# Set up

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.3.0+cu121


In [ ]:
# mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install tqdm
!pip install torch

# Required versions for plotting networkx
!pip install 'networkx<3.0'
!pip install 'scipy>=1.8'

# Random walk library
!pip install csrgraph

# Dataset Prep
!pip install DeepRobust
!pip install scipy

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
 

In [ ]:
class Arguments:
  def __init__(self):
    self.seed = 123
    self.dataset = 'cora'
    self.model_lr = 0.01
    self.weight_decay = 5e-4
    self.hidden_layers = 32
    self.dropout = 0.5
    self.protect_size = 0.1
    self.ptb_rate = 0.25
    self.do_sampling = 'Y'
    self.sample_size = 500
    self.num_samples = 20
    self.ptb_epochs = 30
    self.reg_epochs = 100
args = Arguments()

In [ ]:
!pip install torch -q

In [ ]:
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html


Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.3.0+cu121


In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric

from tqdm import tqdm
import random
import math

# libaries for GNN
from torch_geometric.nn import DenseGCNConv

In [ ]:
# Optional libraries for plotting
import networkx as nx
from networkx.convert import to_networkx_graph
from networkx.generators.classic import Graph
from torch_geometric.utils import to_networkx

# Random Walk library
import csrgraph as cg

In [ ]:
from torch_geometric.nn import DenseGCNConv, GCNConv

In [ ]:
################################################
# Environment
###############################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("cpu")
np.random.seed(args.seed)
torch.manual_seed(args.seed)

if device != 'cpu':
    torch.cuda.manual_seed(args.seed)

print('==== Environment ====')
print(f'  torch version: {torch.__version__}')
print(f'  device: {device}')
print(f'  torch seed: {args.seed}')

==== Environment ====
  torch version: 2.3.0+cu121
  device: cpu
  torch seed: 123


# Utils Methods


## Utils

In [ ]:
class Utils:
  def idx_to_bool(idx, max_len=None):
    """
    Converts an array of indices into a boolean array (where desired indices are True)
    """

    if not max_len:
        max_len = max(idx) + 1
    arr = torch.zeros(max_len)
    arr[idx] = 1
    return arr > 0

  def get_modified_adj(adj, perturbations):
    """
    Inverts the adjacency matrix by a perturbation matrix (where 1 is to perturb, 0 is to not perturb)
    Uses only the bottom triangle of the perturbation matrix
    """
    tri = (adj + perturbations) - torch.mul(adj * perturbations, 2)
    return tri

  def make_symmetric(adj):
    """
    Makes adj. matrix symmetric about the diagonal and sets the diagonal to 0.
    Keeps the upper triangle.
    """
    upper = torch.triu(adj)

    lower = torch.rot90(torch.flip(
        torch.triu(adj, diagonal=1), [0]), 3, [0, 1])

    result = (upper + lower).fill_diagonal_(0)
    return result

  def to_edges(adj):
    """
    Converts an adjacency matrix to a list of edges
    """
    res = torch.triu(adj).float().nonzero().permute(1, 0)
    return res

  def bool_to_idx(bool_list):
    """
    Converts a boolean array (where desired indices are True) into an array of indices
    """
    return bool_list.nonzero()

  def get_modified_adj(adj, perturbations):
    """
    Inverts the adjacency matrix by a perturbation matrix (where 1 is to perturb, 0 is to not perturb)
    Uses only the bottom triangle of the perturbation matrix
    """

    tri = (adj + perturbations) - torch.mul(adj * perturbations, 2)
    return tri

  def projection(perturbations, n_perturbations):
    """
    Get the projection of a perturbation matrix such that the sum over the distribution of perturbations is n_perturbations

    Parameters
    ---
    perturbations : torch.tensor
        probability distribution of perturbations
    n_perturbations : int
        desired number of perturbations

    Returns
    ---
    out : torch.tensor
        projected perturbation matrix

    Examples
    ---
    >>>example

    """

    def bisection(perturbations, a, b, n_perturbations, epsilon):

          def func(perturbations, x, n_perturbations):
            return torch.clamp(perturbations-x, 0, 1).sum() - n_perturbations

          miu = a
          while ((b-a) >= epsilon):
            miu = (a+b)/2
            # Check if middle point is root
            if (func(perturbations, miu, n_perturbations) == 0.0):
                break
            # Decide the side to repeat the steps
            if (func(perturbations, miu, n_perturbations)*func(perturbations, a, n_perturbations) < 0):
                b = miu
            else:
                a = miu
        # print("The value of root is : ","%.4f" % miu)
          return miu

    # projected = torch.clamp(self.adj_changes, 0, 1)
    if torch.clamp(perturbations, 0, 1).sum() > n_perturbations:
          left = (perturbations - 1).min()
          right = perturbations.max()
          miu = bisection(perturbations, left, right, n_perturbations, epsilon=1e-5)
          perturbations.data.copy_(torch.clamp(
              perturbations.data - miu, min=0, max=1))
    else:
          perturbations.data.copy_(torch.clamp(
              perturbations.data, min=0, max=1))

    return perturbations

## Sampling Matrix

In [ ]:
class SamplingMatrix:
    def __init__(self, g0, gX, adj, sample_size=3):
        '''
        input: g0 -> boolean tensor denoting protected set of nodes [num_nodes],
                gX -> boolean tensor denoting authorized set of nodes [number of nodes],
                adj -> tensor for adjacency matrix [num_nodes, num_nodes],
                sample_size -> size of sample to be taken [int]

        function: creates tensor of indices for the protected [len(gX)] and authorized [len(g0)] set of nodes,
                  creates symmetric sampling matrix tensor [num_nodes, num_nodes] for both protected and authorized set where value at a given idx
                    is 1 if the node is a member of the prescribed set and zero if it is not (similar to adjacency matrix)
                  calls updateSamplingMatrix method
        '''
        self.prev_sampling_matrix = torch.tensor(1)

        self.g0_ratio = torch.tensor(1)
        self.gX_ratio = torch.tensor(1)
        self.g0gX_ratio = torch.tensor(1)
        self.sample_size = sample_size

        self.g0 = g0
        self.g0_idx = Utils.bool_to_idx(g0).t()
        self.gX = gX
        self.gX_idx = Utils.bool_to_idx(gX).t()

        self.g0_sampling = torch.zeros_like(adj)
        ####
        self.g0_sampling.index_fill_(0, Utils.bool_to_idx(g0).squeeze(), 1)
        self.g0_sampling.index_fill_(1, Utils.bool_to_idx(gX).squeeze(), 0)
        self.g0_sampling.fill_diagonal_(0)

        self.gX_sampling = torch.zeros_like(adj)
        self.gX_sampling.index_fill_(0, Utils.bool_to_idx(gX).squeeze(), 1)
        self.gX_sampling.index_fill_(1, Utils.bool_to_idx(g0).squeeze(), 0)
        self.gX_sampling.fill_diagonal_(0)

        self.g0gX_sampling = torch.ones_like(adj)
        self.g0gX_sampling -= self.g0_sampling + self.gX_sampling
        self.g0gX_sampling.fill_diagonal_(0)

        self.updateSamplingMatrix()

    def getRatio(self):
        '''
        sums the ratios of g0, gX, and g0gX
        '''
        total = self.g0_ratio + self.gX_ratio + self.g0gX_ratio
        print(f"G0:  {self.g0_ratio / total:.2f}")
        print(f"GX:  {self.gX_ratio / total:.2f}")
        print(f"G0GX:{self.g0gX_ratio / total:.2f}")

    def updateSamplingMatrix(self):
        try:
          self.prev_sampling_matrix = self.sampling_matrix
        except:
          print('first run')

        total = self.g0_ratio + self.gX_ratio + self.g0gX_ratio
        constant_g0 =       ((self.g0_ratio * self.sample_size * 2) / (total * self.g0_sampling.sum()))
        constant_gX =       ((self.gX_ratio * self.sample_size * 2) / (total * self.gX_sampling.sum()))
        constant_g0gX =    ((self.g0gX_ratio * self.sample_size * 2) / (total * self.g0gX_sampling.sum()))

        self.sampling_matrix = \
            self.g0gX_sampling * constant_g0gX + \
            self.g0_sampling * constant_g0 + \
            self.gX_sampling * constant_gX

        self.sampling_matrix = torch.clamp(self.sampling_matrix, min=0, max=1)

        self.sampling_matrix.triu_(diagonal=1)
        # self.sampling_matrix.fill_diagonal_(0)

    def get_sample(self):
        """
        input: self -> sampling matrix [num nodes, num nodes]

        draws random binary numbers using sampling_matrix values as probability distribution /
          converts matrix to a list of edges

        output: edgelist of size [2, ~0.20 * num nodes]
        """
        if torch.isnan(self.sampling_matrix).any():
          self.handle_nan_values(replace_with=1)
        return Utils.to_edges(torch.bernoulli(self.sampling_matrix))

    def updateRatio(self, g0_ratio, gX_ratio, g0gX_ratio):
        """
        input: g0_ratio -> tensor value of ratio of g0-g0 edge in sampling matrix
                gX_ratio -> tensor value of ratio of gX-gX edge in sampling matrix
                g0gX_ratio ->tensor value of ratio of g0-gX edge in sampling matrix

        output: changes ratio values and updates sampling matrix according to input
        """
        self.g0_ratio = g0_ratio
        self.gX_ratio = gX_ratio
        self.g0gX_ratio = g0gX_ratio
        total = self.g0_ratio + self.gX_ratio + self.g0gX_ratio
        self.g0_ratio /= total
        self.gX_ratio /= total
        self.g0gX_ratio /= total

        self.updateSamplingMatrix()

    def updateByGrad(self, adj_grad, count):
        """
        input: adj_grad -> gradient values for adjacency matrix [num nodes, num nodes],
                count -> number of times a given idx has been sampled by get_ratio method [num nodes, num nodes]

        output: sums the sampling matrix values after multiplying by count and updates the
                  ratio values of all sets (g0, gX, g0gX) in the sampling matrix
        """
        min_sample = self.sample_size / 10

        g0r_count = (count * self.g0_sampling).sum() + min_sample
        gXr_count = (count * self.gX_sampling).sum() + min_sample
        g0gXr_count = count.sum() - (g0r_count + gXr_count) + min_sample

        abs_grad = adj_grad.abs()
        g0r = (abs_grad * self.g0_sampling).sum() / g0r_count
        gXr = (abs_grad * self.gX_sampling).sum() / gXr_count
        g0gXr = (abs_grad.sum() - (g0r + gXr)) / g0gXr_count

        # abs_grad = adj_grad.abs()
        # g0r = (abs_grad * self.g0_sampling)
        # g0r = torch.median(g0r[g0r.nonzero()])

        # gXr = (abs_grad * self.gX_sampling)
        # gXr = torch.median(gXr[gXr.nonzero()])

        # g0gXr = (abs_grad * self.g0gX_sampling)
        # g0gXr = torch.median(g0gXr[g0gXr.nonzero()])

        # print(g0r, gXr, g0gXr)
        # print(g0r_count, gXr_count, g0gXr_count)

        total = g0r + gXr + g0gXr
        g0r /= total
        gXr /= total
        g0gXr /= total

        self.updateRatio(
            g0_ratio=(self.g0_ratio + g0r) / 2,
            gX_ratio=(self.gX_ratio + gXr) / 2,
            g0gX_ratio=(self.g0gX_ratio + g0gXr) / 2
        )

    def handle_nan_values(self, replace_with=1):
      """
      replace nan values in sampling matrix with 1 or previous value
      """
      if replace_with != 1:
        idx = Utils.bool_to_idx(torch.isnan(self.sampling_matrix))
        self.sampling_matrix[idx] = self.prev_sampling_matrix[idx]
      else:
        self.sampling_matrix = torch.nan_to_num(self.sampling_matrix, 1.0)

# Get Data

In [ ]:
import scipy.sparse as sp
import os.path as osp
import urllib.request
import sys
import pickle as pkl
import zipfile
import json
from sklearn.model_selection import train_test_split


class Dataset():
    """Dataset class contains four citation network datasets "cora", "cora-ml", "citeseer" and "pubmed",
    and one blog dataset "Polblogs". Datasets "ACM", "BlogCatalog", "Flickr", "UAI",
    "Flickr" are also available. See more details in https://github.com/DSE-MSU/DeepRobust/tree/master/deeprobust/graph#supported-datasets.
    The 'cora', 'cora-ml', 'polblogs' and 'citeseer' are downloaded from https://github.com/danielzuegner/gnn-meta-attack/tree/master/data, and 'pubmed' is from https://github.com/tkipf/gcn/tree/master/gcn/data.
    Parameters
    ----------
    root : string
        root directory where the dataset should be saved.
    name : string
        dataset name, it can be chosen from ['cora', 'citeseer', 'cora_ml', 'polblogs',
        'pubmed', 'acm', 'blogcatalog', 'uai', 'flickr']
    setting : string
        there are three data splits settings. It can be chosen from ['nettack', 'gcn', 'prognn', 'traditional']
        The 'nettack' setting follows nettack paper where they select the largest connected
        components of the graph and use 10%/10%/80% nodes for training/validation/test .
        The 'gcn' setting follows gcn paper where they use the full graph and 20 samples
        in each class for traing, 500 nodes for validation, and 1000
        nodes for test. (Note here 'netack' and 'gcn' setting do not provide fixed split, i.e.,
        different random seed would return different data splits)
        The 'traditional' setting uses a 0.7/0.2/0.1 fixed split for the train/val/test sets respectively. An
        approximately equal ratio of all classes is used per set.
    seed : int
        random seed for splitting training/validation/test.
    require_mask : bool
        setting require_mask True to get training, validation and test mask
        (self.train_mask, self.val_mask, self.test_mask)
    Examples
    --------
	We can first create an instance of the Dataset class and then take out its attributes.
	>>> from deeprobust.graph.data import Dataset
	>>> data = Dataset(root='/tmp/', name='cora', seed=15)
	>>> adj, features, labels = data.adj, data.features, data.labels
	>>> idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
    """

    def __init__(self, root, name, setting='gcn', seed=None, require_mask=False):
        self.name = name.lower()
        self.setting = setting.lower()

        assert self.name in ['cora', 'citeseer', 'cora_ml', 'polblogs',
                'pubmed', 'acm', 'blogcatalog', 'uai', 'flickr'], \
                'Currently only support cora, citeseer, cora_ml, ' + \
                'polblogs, pubmed, acm, blogcatalog, flickr'
        assert self.setting in ['gcn', 'nettack', 'prognn', 'traditional', 'realistic'], "Settings should be" + \
                        " choosen from ['gcn', 'nettack', 'prognn', 'traditional', 'realistic']"

        self.seed = seed
        # self.url =  'https://raw.githubusercontent.com/danielzuegner/nettack/master/data/%s.npz' % self.name
        self.url =  'https://raw.githubusercontent.com/danielzuegner/gnn-meta-attack/master/data/%s.npz' % self.name
        self.root = osp.expanduser(osp.normpath(root))
        self.data_folder = osp.join(root, self.name)
        self.data_filename = self.data_folder + '.npz'
        self.require_mask = require_mask

        self.require_lcc = False if setting == 'gcn' else True
        self.adj, self.features, self.labels = self.load_data()

        if setting == 'prognn':
            assert name in ['cora', 'citeseer', 'pubmed', 'cora_ml', 'polblogs'], "ProGNN splits only " + \
                        "cora, citeseer, pubmed, cora_ml, polblogs"
            self.idx_train, self.idx_val, self.idx_test = self.get_prognn_splits()
        else:
            self.idx_train, self.idx_val, self.idx_test = self.get_train_val_test()
        if self.require_mask:
            self.get_mask()

    def get_train_val_test(self):
        """Get training, validation, test splits according to self.setting (either 'nettack' or 'gcn' or 'traditional').
        """
        if self.setting == 'nettack':
            return get_train_val_test(nnodes=self.adj.shape[0], val_size=0.1, test_size=0.8, stratify=self.labels, seed=self.seed)
        if self.setting == 'gcn':
            return get_train_val_test_gcn(self.labels, seed=self.seed)
        if self.setting == 'traditional':
            return get_train_val_test_trad(self.labels, seed=self.seed)
        if self.setting == 'realistic':
            return get_train_val_test_real(self.labels, seed=self.seed)

    def get_prognn_splits(self):
        """Get target nodes incides, which is the nodes with degree > 10 in the test set."""
        url = 'https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/' + \
                     'master/splits/{}_prognn_splits.json'.format(self.name)
        json_file = osp.join(self.root,
                '{}_prognn_splits.json'.format(self.name))

        if not osp.exists(json_file):
            self.download_file(url, json_file)
        # with open(f'/mnt/home/jinwei2/Projects/nettack/{dataset}_nettacked_nodes.json', 'r') as f:
        with open(json_file, 'r') as f:
            idx = json.loads(f.read())
        return np.array(idx['idx_train']), \
               np.array(idx['idx_val']), np.array(idx['idx_test'])

    def load_data(self):
        print('Loading {} dataset...'.format(self.name))
        if self.name == 'pubmed':
            return self.load_pubmed()

        if self.name in ['acm', 'blogcatalog', 'uai', 'flickr']:
            return self.load_zip()

        if not osp.exists(self.data_filename):
            self.download_npz()

        adj, features, labels = self.get_adj()
        return adj, features, labels

    def download_file(self, url, file):
        print('Dowloading from {} to {}'.format(url, file))
        try:
            urllib.request.urlretrieve(url, file)
        except:
            raise Exception("Download failed! Make sure you have \
                    stable Internet connection and enter the right name")

    def download_npz(self):
        """Download adjacen matrix npz file from self.url.
        """
        print('Downloading from {} to {}'.format(self.url, self.data_filename))
        try:
            urllib.request.urlretrieve(self.url, self.data_filename)
            print('Done!')
        except:
            raise Exception('''Download failed! Make sure you have stable Internet connection and enter the right name''')

    def download_pubmed(self, name):
        url = 'https://raw.githubusercontent.com/tkipf/gcn/master/gcn/data/'
        try:
            print('Downloading', url)
            urllib.request.urlretrieve(url + name, osp.join(self.root, name))
            print('Done!')
        except:
            raise Exception('''Download failed! Make sure you have stable Internet connection and enter the right name''')

    def download_zip(self, name):
        url = 'https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/master/other_datasets/{}.zip'.\
                format(name)
        try:
            print('Downlading', url)
            urllib.request.urlretrieve(url, osp.join(self.root, name+'.zip'))
            print('Done!')
        except:
            raise Exception('''Download failed! Make sure you have stable Internet connection and enter the right name''')

    def load_zip(self):
        data_filename = self.data_folder + '.zip'
        name = self.name
        if not osp.exists(data_filename):
            self.download_zip(name)
            with zipfile.ZipFile(data_filename, 'r') as zip_ref:
                zip_ref.extractall(self.root)

        feature_path = osp.join(self.data_folder, '{0}.feature'.format(name))
        label_path = osp.join(self.data_folder, '{0}.label'.format(name))
        graph_path = osp.join(self.data_folder, '{0}.edge'.format(name))

        f = np.loadtxt(feature_path, dtype = float)
        l = np.loadtxt(label_path, dtype = int)
        features = sp.csr_matrix(f, dtype=np.float32)
        # features = torch.FloatTensor(np.array(features.todense()))
        struct_edges = np.genfromtxt(graph_path, dtype=int)
        sedges = np.array(list(struct_edges), dtype=int).reshape(struct_edges.shape)
        n = features.shape[0]
        sadj = sp.coo_matrix((np.ones(sedges.shape[0]), (sedges[:, 0], sedges[:, 1])), shape=(n, n), dtype=np.float32)
        sadj = sadj + sadj.T.multiply(sadj.T > sadj) - sadj.multiply(sadj.T > sadj)
        label = np.array(l)

        return sadj, features, label

    def load_pubmed(self):
        dataset = 'pubmed'
        names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
        objects = []
        for i in range(len(names)):
            name = "ind.{}.{}".format(dataset, names[i])
            data_filename = osp.join(self.root, name)

            if not osp.exists(data_filename):
                self.download_pubmed(name)

            with open(data_filename, 'rb') as f:
                if sys.version_info > (3, 0):
                    objects.append(pkl.load(f, encoding='latin1'))
                else:
                    objects.append(pkl.load(f))

        x, y, tx, ty, allx, ally, graph = tuple(objects)


        test_idx_file = "ind.{}.test.index".format(dataset)
        if not osp.exists(osp.join(self.root, test_idx_file)):
            self.download_pubmed(test_idx_file)

        test_idx_reorder = parse_index_file(osp.join(self.root, test_idx_file))
        test_idx_range = np.sort(test_idx_reorder)

        features = sp.vstack((allx, tx)).tolil()
        features[test_idx_reorder, :] = features[test_idx_range, :]
        adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
        labels = np.vstack((ally, ty))
        labels[test_idx_reorder, :] = labels[test_idx_range, :]
        labels = np.where(labels)[1]
        return adj, features, labels

    def get_adj(self):
        adj, features, labels = self.load_npz(self.data_filename)
        adj = adj + adj.T
        adj = adj.tolil()
        adj[adj > 1] = 1

        if self.require_lcc:
            lcc = self.largest_connected_components(adj)
            adj = adj[lcc][:, lcc]
            features = features[lcc]
            labels = labels[lcc]
            assert adj.sum(0).A1.min() > 0, "Graph contains singleton nodes"

        # whether to set diag=0?
        adj.setdiag(0)
        adj = adj.astype("float32").tocsr()
        adj.eliminate_zeros()

        assert np.abs(adj - adj.T).sum() == 0, "Input graph is not symmetric"
        assert adj.max() == 1 and len(np.unique(adj[adj.nonzero()].A1)) == 1, "Graph must be unweighted"

        return adj, features, labels

    def load_npz(self, file_name, is_sparse=True):
        with np.load(file_name) as loader:
            # loader = dict(loader)
            if is_sparse:
                adj = sp.csr_matrix((loader['adj_data'], loader['adj_indices'],
                                            loader['adj_indptr']), shape=loader['adj_shape'])
                if 'attr_data' in loader:
                    features = sp.csr_matrix((loader['attr_data'], loader['attr_indices'],
                                                 loader['attr_indptr']), shape=loader['attr_shape'])
                else:
                    features = None
                labels = loader.get('labels')
            else:
                adj = loader['adj_data']
                if 'attr_data' in loader:
                    features = loader['attr_data']
                else:
                    features = None
                labels = loader.get('labels')
        if features is None:
            features = np.eye(adj.shape[0])
        features = sp.csr_matrix(features, dtype=np.float32)
        return adj, features, labels

    def largest_connected_components(self, adj, n_components=1):
        """Select k largest connected components.
		Parameters
		----------
		adj : scipy.sparse.csr_matrix
			input adjacency matrix
		n_components : int
			n largest connected components we want to select
		"""

        _, component_indices = sp.csgraph.connected_components(adj)
        component_sizes = np.bincount(component_indices)
        components_to_keep = np.argsort(component_sizes)[::-1][:n_components]  # reverse order to sort descending
        nodes_to_keep = [
            idx for (idx, component) in enumerate(component_indices) if component in components_to_keep]
        print("Selecting {0} largest connected components".format(n_components))
        return nodes_to_keep

    def __repr__(self):
        return '{0}(adj_shape={1}, feature_shape={2})'.format(self.name, self.adj.shape, self.features.shape)

    def get_mask(self):
        idx_train, idx_val, idx_test = self.idx_train, self.idx_val, self.idx_test
        labels = self.onehot(self.labels)

        def get_mask(idx):
            mask = np.zeros(labels.shape[0], dtype=np.bool)
            mask[idx] = 1
            return mask

        def get_y(idx):
            mx = np.zeros(labels.shape)
            mx[idx] = labels[idx]
            return mx

        self.train_mask = get_mask(self.idx_train)
        self.val_mask = get_mask(self.idx_val)
        self.test_mask = get_mask(self.idx_test)
        self.y_train, self.y_val, self.y_test = get_y(idx_train), get_y(idx_val), get_y(idx_test)

    def onehot(self, labels):
        eye = np.identity(labels.max() + 1)
        onehot_mx = eye[labels]
        return onehot_mx

def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def get_train_val_test(nnodes, val_size=0.1, test_size=0.8, stratify=None, seed=None):
    """This setting follows nettack/mettack, where we split the nodes
    into 10% training, 10% validation and 80% testing data
    Parameters
    ----------
    nnodes : int
        number of nodes in total
    val_size : float
        size of validation set
    test_size : float
        size of test set
    stratify :
        data is expected to split in a stratified fashion. So stratify should be labels.
    seed : int or None
        random seed
    Returns
    -------
    idx_train :
        node training indices
    idx_val :
        node validation indices
    idx_test :
        node test indices
    """

    assert stratify is not None, 'stratify cannot be None!'

    if seed is not None:
        np.random.seed(seed)

    idx = np.arange(nnodes)
    train_size = 1 - val_size - test_size
    idx_train_and_val, idx_test = train_test_split(idx,
                                                   random_state=None,
                                                   train_size=train_size + val_size,
                                                   test_size=test_size,
                                                   stratify=stratify)

    if stratify is not None:
        stratify = stratify[idx_train_and_val]

    idx_train, idx_val = train_test_split(idx_train_and_val,
                                          random_state=None,
                                          train_size=(train_size / (train_size + val_size)),
                                          test_size=(val_size / (train_size + val_size)),
                                          stratify=stratify)


def get_train_val_test_gcn(labels, seed=None):
    """This setting follows gcn, where we randomly sample 20 instances for each class
    as training data, 500 instances as validation data, 1000 instances as test data.
    Note here we are not using fixed splits. When random seed changes, the splits
    will also change.
    Parameters
    ----------
    labels : numpy.array
        node labels
    seed : int or None
        random seed
    Returns
    -------
    idx_train :
        node training indices
    idx_val :
        node validation indices
    idx_test :
        node test indices
    """

    if seed is not None:
        np.random.seed(seed)

    idx = np.arange(len(labels))
    nclass = labels.max() + 1
    idx_train = []
    idx_unlabeled = []
    for i in range(nclass):
        labels_i = idx[labels==i]
        labels_i = np.random.permutation(labels_i)
        idx_train = np.hstack((idx_train, labels_i[: 20])).astype(int)
        idx_unlabeled = np.hstack((idx_unlabeled, labels_i[20: ])).astype(int)

    idx_unlabeled = np.random.permutation(idx_unlabeled)
    idx_val = idx_unlabeled[: 500]
    idx_test = idx_unlabeled[500: 1500]
    return idx_train, idx_val, idx_test


def get_train_val_test_trad(labels, seed=None):
    """Splits data using a traditional fixed split of 0.7/0.2/0.1 for train/val/test
    sets respectively. Each class is distributed so that each set contains
    an approximately equal ratio of nodes per class.
    Parameters
    ----------
    labels : numpy.array
        node labels
    seed : int or None
        random seed
    Returns
    -------
    idx_train :
        node training indices
    idx_val :
        node validation indices
    idx_test :
        node test indices
    """

    if seed is not None:
        np.random.seed(seed)

    nclass = labels.max() + 1
    num_nodes = len(labels)
    train_split = math.floor((0.7 * num_nodes) / (nclass))
    val_split = math.floor((0.2 * num_nodes) / (nclass))
    test_split = math.floor((0.1 * num_nodes) / (nclass))

    idx = np.arange(len(labels))
    idx_train = []
    idx_val = []
    idx_test = []
    idx_unlabeled = []
    for i in range(nclass):
        labels_i = idx[labels==i]
        labels_i = np.random.permutation(labels_i)
        idx_train = np.hstack((idx_train, labels_i[: train_split])).astype(int)
        idx_val = np.hstack((idx_val, labels_i[train_split : train_split + val_split])).astype(int)
        idx_test = np.hstack((idx_test, labels_i[train_split + val_split : train_split + val_split + test_split])).astype(int)

    return idx_train, idx_val, idx_test


def get_train_val_test_real(labels, seed=None):
    """Splits data using a fixed split of 0.7/0.2/0.1 for train/val/test
    sets respectively. Each class is distributed so that each set contains
    a ratio of classes ~equal to class ratio in the entire dataset.
    Parameters
    ----------
    labels : numpy.array
        node labels
    seed : int or None
        random seed
    Returns
    -------
    idx_train :
        node training indices
    idx_val :
        node validation indices
    idx_test :
        node test indices
    """

    nclass = labels.max() + 1
    num_nodes = len(labels)
    train_split = math.floor((0.7 * num_nodes))
    val_split = math.floor((0.2 * num_nodes))
    test_split = math.floor((0.1 * num_nodes))

    idx = np.arange(len(labels))
    idx_train = []
    idx_val = []
    idx_test = []
    idx_unlabeled = []
    for i in range(nclass):
        ratio = sum(labels==i) / len(labels)
        r_train = math.floor(train_split * ratio)
        r_val = math.floor(val_split * ratio)
        r_test = math.floor(test_split * ratio)
        labels_i = idx[labels==i]
        labels_i = np.random.permutation(labels_i)
        idx_train = np.hstack((idx_train, labels_i[: r_train])).astype(int)
        idx_val = np.hstack((idx_val, labels_i[r_train : r_train + r_val])).astype(int)
        idx_test = np.hstack((idx_test, labels_i[r_train + r_val : r_train + r_val + r_test])).astype(int)

    return idx_train, idx_val, idx_test


In [ ]:
class Graph:
    def __init__(self, adj, labels, features, idx_train, idx_val, idx_test, split_seed, device):
        self.adj = adj.to(device)
        self.features = features.to(device)
        self.labels = labels.to(device)
        self.idx_train = idx_train.to(device)
        self.idx_val = idx_val.to(device)
        self.idx_test = idx_test.to(device)
        self.split_seed = split_seed

    def __repr__(self):
        return f"<Graph {self.adj.shape[0]}x{self.adj.shape[1]}>"

    def summarize(self, name=""):
        print()
        print(f'[i] Dataset Summary: {name}')
        print(f'\tadj shape: {list(self.adj.shape)}')
        print(f'\tfeature shape: {list(self.features.shape)}')
        print(f'\tnum labels: {self.labels.max().item()+1}')
        print(f'\tsplit seed: {self.split_seed}')
        print(
            f'\ttrain|val|test: {self.idx_train.sum()}|{self.idx_val.sum()}|{self.idx_test.sum()}')

    def split(self, nsplits):
        indices = torch.zeros(10, dtype=torch.bool)
        return None

    def numEdges(self):
        return self.adj.sum() / 2

    def adjacency(self):
        return self.adj

    def numNodes(self):
        return self.adj.shape[0]

    def getSample(self, size):
        indices = (torch.bernoulli(torch.empty(1, size)[0].uniform_(0,1))) > 0.5
        maskA = indices.nonzero().t()[0]
        maskB = (~indices).nonzero().t()[0]

        return maskA, maskB

    def getSubgraph(self, indices):
        return Graph(
            adj=self.adj[indices].t()[indices].t(),
            features=self.features[indices],
            labels=self.labels[indices],
            idx_train=self.idx_train[indices],
            idx_val=self.idx_val[indices],
            idx_test=self.idx_test[indices],
            device=self.device
        )

    def getEntropies(self):
        # TODO
        return 0

def getGraph(root, name, setting, seed, device, verbose=True):
    data = Dataset(root, name, setting, seed)

    adj = torch.LongTensor(data.adj.todense())
    adj = Utils.make_symmetric(adj)
    labels = torch.LongTensor(data.labels)
    features = torch.FloatTensor(np.array(data.features.todense()))

    def indices_to_bool(indices, length):
        arr = torch.zeros(length)
        arr[indices] = 1
        return arr > 0

    idx_train = Utils.idx_to_bool(data.idx_train, features.shape[0])
    idx_val = Utils.idx_to_bool(data.idx_val, features.shape[0])
    idx_test = Utils.idx_to_bool(data.idx_test, features.shape[0])

    return Graph(adj, labels, features, idx_train, idx_val, idx_test, seed, device)

# Dataset


In [ ]:
args.dataset = 'cora'

print(f'==== Dataset: {args.dataset} ====')
graph = getGraph(root='.', name=args.dataset, setting='gcn', seed=args.seed, device=device)

# CiteSeer: worse initial accuracy, C&S hyperparameters had to be lowered in order to run
# flickr: initial acc ~50%
# BlogCatalog
# cora: initial acc ~79%
# PolBlogs: initial acc ~86%

==== Dataset: cora ====
Loading cora dataset...
Done!


In [ ]:
graph.num_nodes = graph.numNodes() #graph.features.shape[0]
graph.num_features = graph.features.shape[1]
graph.num_edges = graph.numEdges() #graph.edge_index.shape[1]
graph.device = device
graph.edge_index = graph.adj.nonzero().t().contiguous()

In [ ]:
## NetworkX Graph functionality ##
from torch_geometric.data import Data

data = Data(x=graph.features, y=graph.labels, edge_index=graph.edge_index)

nxg = nx.Graph()
nxg = to_networkx(data, to_undirected=True)
graph.nx_g = nxg

graph.clustering = nx.clustering(nxg)
graph.centrality = nx.degree_centrality(nxg)
graph.degree = nxg.degree

In [ ]:
nx.is_connected(graph.nx_g)

False

## Adjacency matrix

In [ ]:
def to_sym_adj(edge_index, num_nodes):
  '''
  input: edge_index is tensor of shape [2, num_edges] and num_nodes is integer
  output: tensors of shape [num_nodes, num_nodes] that represents the symmetric adjaceny matrix of edge_index
  '''
  num_edges = graph.edge_index.shape[1]
  adj = torch.zeros(num_nodes, num_nodes )
  for i in range(num_edges):
    u, v = graph.edge_index[0][i],  graph.edge_index[1][i]
    adj[u][v] = 1
    adj[v][u] = 1
  return adj

def to_edge_index(adj_matrix):
  '''
  input: adj matrix tensor of shape [num_nodes, num_nodes]
  output: tensor of shape [2, num_edges]
  '''
  edge_index = graph.adj.nonzero().t().contiguous()
  return edge_index

In [ ]:
graph.adj = to_sym_adj(graph.edge_index, graph.num_nodes).to(device)

In [ ]:
# sanity check
from torch_geometric.utils import to_dense_adj
adj_edge_index = to_dense_adj(graph.edge_index).squeeze()
(adj_edge_index == graph.adj).all()

tensor(True)

In [ ]:
# Designate protected nodes g0
g0 = torch.rand(graph.numNodes()) <= args.protect_size
g0 = g0.to(device)
gX = ~g0.to(device)

print(f"Number of protected nodes: {g0.sum():.0f}")
print(f"Ratio of protected nodes: {g0.sum() / graph.features.shape[0]:.2%}")

# Sampling matrix
samplingMatrix = SamplingMatrix(g0, gX, graph.adj, args.sample_size)
print((samplingMatrix.sampling_matrix))

Number of protected nodes: 285
Ratio of protected nodes: 10.52%
first run
tensor([[0.0000e+00, 5.6800e-05, 5.6800e-05,  ..., 5.6800e-05, 5.6800e-05,
         5.6800e-05],
        [0.0000e+00, 0.0000e+00, 5.6800e-05,  ..., 5.6800e-05, 5.6800e-05,
         5.6800e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.6800e-05, 5.6800e-05,
         5.6800e-05],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 5.6800e-05,
         5.6800e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         5.6800e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


In [ ]:
# (samplingMatrix.sampling_matrix>=0).all() and (samplingMatrix.sampling_matrix<=1).all()

In [ ]:
#torch.bernoulli?

In [ ]:
#samplingMatrix.get_sample()

In [ ]:
#samplingMatrix.getRatio()

# Metrics Methods

In [ ]:

class Metrics:
  def acc(predictions, labels):
    """
    input: float tensor of predicted values of nodes [num_nodes, num_classes], int tensor of ground truth labels of nodes [num_nodes]

    output: accuracy value of predictions calcualted by dividing the number of correct predictions by total number of predictions
    """
    correct = (predictions.argmax(1) == labels).sum()
    acc = correct / predictions.size(dim=0)
    return acc.item()

  def calc_acc(model, features, adj, labels, idx=False):
    """
    input: GNN model, tensor of graph features[num_nodes, num_features], adj matrix[num_nodes, num_nodes],
              tensor of graph labels[num_nodes], desired indices to compare truth values[num_nodes]

    output: calculated acc of correctly predicted nodes (float)
    """
    if not idx:
        idx = torch.ones_like(labels) > 0

    pred = model(features, adj)

    correct = (pred.argmax(1)[idx] == labels[idx]).sum()
    acc = correct / idx.sum()
    return acc.item()

  def partial_acc(predictions, labels, g0, g_g0, verbose=True):
    """
    input: float tensor predicted values of nodes [num_nodes, num_classes], int tensor of ground truth label of nodes [num_nodes],
       boolean tensor of denoting authorized set of nodes [num_nodes], boolean tensor denoting protected set of nodes [number of nodes]

    output: dict with calculated accuracy values of both the authorized and protected set of nodes
    """
    g0_acc = Metrics.acc(predictions[g0], labels[g0])
    gX_acc = Metrics.acc(predictions[g_g0], labels[g_g0])

    if verbose:
        print(f"G0: {g0_acc:.2%}")
        print(f"GX: {gX_acc:.2%}")

    return {
        "g0": g0_acc,
        "gX": gX_acc
    }

  def mask_adj(adj, bool_list, device):
    """
    input: adj matrix [num_nodes, num_nodes], list of bools where desired indices are true [num_nodes], device architecture [string]

    output: modified adj matrix [num_nodes, num_nodes] of only true values of bool_list
    """
    idx = Utils.bool_to_idx(bool_list).squeeze().to(device)

    temp_adj = adj.clone().to(device)
    temp_adj.index_fill_(dim=0, index=idx, value=0)
    diff = adj - temp_adj

    temp_adj = diff.clone().to(device)
    temp_adj.index_fill_(dim=1, index=idx, value=0)
    diff = diff - temp_adj

    # add = int(diff.clamp(0,1).sum() / 2)
    # remove = int(diff.clamp(-1,0).abs().sum() / 2)

    return diff

  def show_metrics(changes, labels, g0, device, verbose=True):
    """
    Prints the changes in edges with respect to g0 and labels of a diff

    Parameters
    ---
    par_name : par_type
        par_description

    Returns
    ---
    out : ret_type
        ret_description

    Examples
    ---
    >>>example

    """

    def print_same_diff(type, adj):
        edges = Utils.to_edges(adj)
        same = 0
        for edge in edges.t():
            same += int(labels[edge[0]].item() == labels[edge[1]].item())

        diff = edges.shape[1] - same

        if verbose: print(f"     {type}   {int(same)}  \t{int(diff)}  \t{int(same+diff)}")
        return { "same": int(same), "diff": int(diff), "total": int(same + diff)}

    def print_add_remove(adj):
        """prints change in edges"""
        add = adj.clamp(0,1)
        remove = adj.clamp(-1,0).abs()
        if verbose: print("                A-A\tA-B\tTOTAL")
        numAdd = print_same_diff("     (+)", add)
        numRemove = print_same_diff("     (-)", remove)

        return {"add": numAdd, "remove": numRemove, "total": numAdd["total"] + numRemove["total"]}
    # print_add_remove(changes)

    r = {}

    if verbose: print("     Within G0 ====")
    g0_adj = Metrics.mask_adj(changes, g0, device)
    r["g0"] = print_add_remove(g0_adj)

    if verbose: print("     Within GX ====")
    gX_adj = Metrics.mask_adj(changes, ~g0, device)
    r["gX"] = print_add_remove(gX_adj)

    if verbose: print("     Between G0-GX ====")
    g0gX_adj = (changes - g0_adj - gX_adj)
    r["g0gX"] = print_add_remove(g0gX_adj)

    if verbose: print()
    print_same_diff("   TOTAL", changes)

    return r

def calc_entropy(data: torch.tensor, numbins=50):
    bins = torch.histc(data, bins=numbins)
    bins /= bins.sum()
    return ((bins * torch.log2(bins)).nan_to_num().sum() * -1).item()

def calc_correlation(tensor1: torch.tensor, tensor2: torch.tensor):
    """calculates correlation between two tensor objects"""
    cat = torch.cat((tensor1.unsqueeze(0).cpu(), tensor2.unsqueeze(0).cpu())).numpy()
    return np.corrcoef(cat)[0][1]

def get_ent_cor(features: torch.tensor, labels: torch.tensor, num: int=10, rand=False, offset=0):
    """
    Return the features with most entropy and/or correlation

    Parameters
    ---
    par_name : par_type
        par_description

    Returns
    ---
    entropy, correlation, index

    Examples
    ---
    >>>example

    """

    ent_cor = torch.zeros(3, features.shape[1])

    for r in range(features.shape[1]):
        feat = features.t()[r]
        entropy = calc_entropy(feat)
        correlation = abs(calc_correlation(feat, labels))
        ent_cor[0][r] = entropy
        ent_cor[1][r] = correlation
        ent_cor[2][r] = entropy + correlation

    ent_cor.nan_to_num_()

    if rand:
        idx = torch.tensor(random.sample(range(features.shape[1]), num))
    else:
        idx = torch.topk(ent_cor[2], num + offset, sorted=True).indices[offset:]

    data = ent_cor[:,idx]

    return data[0], data[1], idx


# Correct and Smooth


## TODO: Change the implementation of Correct_and_Smooth, variables in init should be parameters (num_correction_layers=50, correction_alpha=1.0,
num_smoothing_layers=50, smoothing_alpha=0.8,
autoscale=False, scale=20.)

In [ ]:
from numpy import linspace

class Correct_and_Smooth():
    def __init__(self, adj, labels, idx_train, idx_val, idx_test):

      self.A = adj
      self.labels = labels
      self.idx_train = idx_train
      self.idx_val = idx_val
      self.idx_test = idx_test
      self.D = torch.diag(self.A.sum(-1))
      self.D_inv_sqrt = self.D.pow(-0.5)

      self.D_inv_sqrt[self.D_inv_sqrt == float('inf')] = 0 #
      self.S = (self.D_inv_sqrt @ self.A @ self.D_inv_sqrt).to(device) # Normalized adj matrix

      self.y = self.labels.squeeze()
      self.Y = F.one_hot(self.y, int(max(self.y) + 1)).to(device)

    def residual_error(self, Z):
      """
      Form residual error matrix E = Z - Y for training data
      """
      E = Z - self.Y
      E[self.idx_test + self.idx_val] = 0
      return E

    def correct(self, E, alpha1 = 0.8, eps = 1e-5, show_progress=True):
      """
      Step 1: Smooth error across the graph
      E^(t+1) = (1-alpha1)E + alpha * S @ E^(t) -> Ehat  # alpha1 is hyperparameter
      """
      itr = 0
      Ehat = E
      diff = eps
      while diff >= eps and itr < 500:
        Et = (1 - alpha1) * E + alpha1 * (self.S @ Ehat) # (S @ Ehat) sets new error for each node as weighted average of neighbors error
        diff = float(torch.norm(Ehat - Et))
        Ehat = Et
        if show_progress:
            print(f"Iteration: [{itr}] - Diff: [{diff}]")
        itr += 1
      return Ehat

    def autoscale(self, E, Ehat, Z): # Size of new errors now in the same scale as original errors
      """
      Step 2:  Adding residual errors back to predictions gives new prediction vector Zr, normalize and re-scale new predictions
      sigma = sum of absolute value of E for each training sample / num training samples
      """
      sigma = float(sum(torch.norm(E[self.idx_train], p=1, dim=-1))) / len(self.idx_train)
      Zr = Z + sigma * Ehat / sum(abs(Ehat))
      Zr[self.idx_train] = Z[self.idx_train]
      return Zr

    def smooth(self, G, alpha2=0.8, eps=1e-5, show_progress=True): # G is current best guess matrix, alpha2 is hyperparameter
      """
      Step 3: Smooth final predictions over our best guess matrix G, initialized to our scale prediction vector
      G^(t+1) = (1 - alpha)G + alpha2 SG^(t) -> Yhat
      """
      itr = 0
      yhat = G
      diff = eps
      while diff >= eps and itr < 500:
        Gt = (1 - alpha2) * G + alpha2 * (self.S @ yhat)
        diff = float(torch.norm(yhat - Gt))
        yhat = Gt
        if show_progress:
            print(f"Iteration: [{itr}] - Diff: [{diff}]")
        itr += 1
      return yhat

    def correct_and_smooth(self, Z, a1=0.85, a2=0.85, eps=1e-6, show_p= False):
      """
      Full pipeline for C&S
      """
      #
      Z = torch.softmax(Z, -1).to(device)

      E = self.residual_error(Z)
      Ehat = self.correct(E, alpha1=a1, eps=eps, show_progress= show_p)
      G = self.autoscale(E, Ehat, Z)
      G[self.idx_train] = self.Y[self.idx_train].type(torch.float32)
      yhat = self.smooth(G, alpha2=a2, eps=eps, show_progress= show_p)
      return yhat

    def hp_sweep(self, Z, hyperparams1, hyperparams2, show_p= False):
      val_idx = Utils.bool_to_idx(self.idx_val)
      E = self.residual_error(Z)
      values = []
      for hp1 in hyperparams1:
        for hp2 in hyperparams2:
          yhat = self.correct_and_smooth(Z, a1=hp1, a2=hp2, show_p= show_p)
          #yhat = torch.argmax(yhat, -1)
          val_acc = Metrics.acc(yhat[self.idx_val], self.y[self.idx_val])
          values.append([val_acc, (hp1, hp2)])
      return sorted(values)

# GCN


## TODO: idx_val is not used?!

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, dim_input_features, dim_output_classes, num_hidden_layers,
        device, lr=0.01, dropout=0.5, weight_decay=5e-4, name=""):

        super(GCN, self).__init__()

        self.conv1 = GCNConv(dim_input_features, num_hidden_layers)
        self.conv2 = GCNConv(num_hidden_layers, dim_output_classes)

        self.lr = lr
        self.dropout = dropout
        self.weight_decay = weight_decay
        self.name = name
        self.device = device
        self.to(device)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1).squeeze()

    def predict(self, x, edge_index, *_):
        if edge_index.shape[0] == graph.numNodes():
          edge_index = to_edge_index(edge_index)

        return self.forward(x, edge_index)


    def fit(self, x, edge_index, labels, idx_train, idx_test, epochs, verbose=True):

        if epochs == 0:
            return None

        self.train()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}', disable=not verbose)
        t.set_description(f"Training {self.name}")

        for epoch in t:
            optimizer.zero_grad()
            predictions = self(x, edge_index)

            loss = F.cross_entropy(predictions[idx_train], labels[idx_train])

            loss.backward()
            optimizer.step()
            t.set_postfix({"loss": round(loss.item(), 2)})
        return predictions

    def train1epoch(self, x, edge_index, labels, idx_train, idx_test):

        self.train()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        optimizer.zero_grad()
        predictions = self(x, edge_index)

        loss = F.cross_entropy(predictions[idx_train], labels[idx_train])

        loss.backward()
        optimizer.step()
        return loss.item()

In [ ]:
# sanity check
gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"gcn"
).to(device)


In [ ]:
pred = gcn.predict(graph.features, graph.adj)
Metrics.partial_acc(pred, graph.labels, g0, gX)

G0: 7.72%
GX: 9.66%


{'g0': 0.07719298452138901, 'gX': 0.09657449275255203}

In [ ]:
#post_pred = gcn.post(graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
#Metrics.partial_acc(post_pred, graph.labels, g0, gX)

# GCN_ADJ

In [ ]:
class GCN_ADJ(torch.nn.Module):
    def __init__(self, dim_input_features, dim_output_classes, num_hidden_layers,
        device, lr=0.01, dropout=0.5, weight_decay=5e-4, name=""):

        super(GCN_ADJ, self).__init__()

        self.conv1 = DenseGCNConv(dim_input_features, num_hidden_layers)
        self.conv2 = DenseGCNConv(num_hidden_layers, dim_output_classes)

        self.lr = lr
        self.dropout = dropout
        self.weight_decay = weight_decay
        self.name = name
        self.device = device
        self.to(device)

    def forward(self, x, adj):
        # x, edge_index = data.x, data.edge_index

        x = self.conv1(x, adj)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.conv2(x, adj)

        return F.log_softmax(x, dim=1).squeeze()

    def predict(self, x, edge_index, *_):
        return self.forward(x, edge_index)

    def fit(self, graph, epochs, verbose=True):

        if epochs == 0:
            return None

        g_features = graph.features
        adj = graph.adj
        g_labels = graph.labels

        self.train()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}', disable=not verbose)
        t.set_description(f"Training {self.name}")

        for epoch in t:
            optimizer.zero_grad()
            predictions = self(g_features, adj)

            loss = F.cross_entropy(predictions[graph.idx_train], g_labels[graph.idx_train])

            loss.backward()
            optimizer.step()
            t.set_postfix({"loss": round(loss.item(), 2)})

        return predictions

    def fitManual(self, features, adj, labels, idx_train, idx_test, epochs, verbose=True):

        if epochs == 0:
            return None

        self.train()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}', disable=not verbose)
        t.set_description(f"Training {self.name}")

        for epoch in t:
            optimizer.zero_grad()
            predictions = self(features, adj)

            loss = F.cross_entropy(predictions[idx_train], labels[idx_train])

            loss.backward()
            optimizer.step()
            t.set_postfix({"loss": round(loss.item(), 2)})

        return predictions

    def train1epoch(self, features, adj, labels, idx_train, idx_test):

        g_features = graph.features
        adj = adj
        g_labels = graph.labels

        self.train()
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        optimizer.zero_grad()
        predictions = self(g_features, adj)

        loss = F.cross_entropy(predictions[idx_train], g_labels[idx_train])

        loss.backward()
        optimizer.step()

        return loss.item()

# GCN_CS

In [ ]:
class GCN_CS(torch.nn.Module):
    def __init__(self, dim_input_features, dim_output_classes, num_hidden_layers,
        device, lr=0.01, dropout=0.5, weight_decay=5e-4, name=""):
      super(GCN_CS, self).__init__()
      self.gcn = GCN_ADJ(dim_input_features, dim_output_classes, num_hidden_layers, device, lr, dropout, weight_decay, name)

    def post(self,x, adj, labels, idx_train, idx_val, idx_test):
        cs = Correct_and_Smooth(adj, labels, idx_train, idx_val, idx_test)
        pred = self.forward(x, adj)
        pred = cs.correct_and_smooth(pred, a1=0.9, a2=0.9, eps=1e-6)
        return pred

    def forward(self, x, adj):
        return self.gcn.forward(x, adj)

    def predict(self, x, adj, labels, idx_train, idx_val, idx_test):
        return self.post(x, adj, labels, idx_train, idx_val, idx_test)

    def fit(self, x, adj, labels, idx_train, idx_test, epochs, verbose=True):
        return self.gcn.fit(x, adj, labels, idx_train, idx_test, epochs)

    def train1epoch(self, x, adj, labels, idx_train, idx_test):
        return self.gcn.train1epoch(x, adj, labels, idx_train, idx_test)

In [ ]:
# sanity check
gcn_cs = GCN_CS(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"gcn_cs"
).to(device)

In [ ]:
#gcn_cs.predict(graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)

In [ ]:
#pred = gcn_cs.predict(graph.features, graph.adj,graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
Metrics.partial_acc(pred, graph.labels, g0, gX)

G0: 7.72%
GX: 9.66%


{'g0': 0.07719298452138901, 'gX': 0.09657449275255203}

# Explainer Utils

In [ ]:
'''train_utils.py
    Some training utilities.
'''
import torch.optim as optim
class train_utils:
    def build_optimizer(args, params, weight_decay=0.0):
        filter_fn = filter(lambda p : p.requires_grad, params)
        if args.opt == 'adam':
            optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
        elif args.opt == 'sgd':
            optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
        elif args.opt == 'rmsprop':
            optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
        elif args.opt == 'adagrad':
            optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
        if args.opt_scheduler == 'none':
            return None, optimizer
        elif args.opt_scheduler == 'step':
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
        elif args.opt_scheduler == 'cos':
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
        return scheduler, optimizer

In [ ]:
!pip install tensorboardX

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)


In [ ]:
"""
    io_utils.py
    Utilities for reading and writing logs.
"""
import statistics
import re
import csv

import numpy as np
import pandas as pd
import scipy as sc


import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import torch
import networkx as nx
import tensorboardX

import cv2

import torch
import torch.nn as nn
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()

class io_utils:
  def gen_prefix(args):
      '''Generate label prefix for a graph model.
      '''
      if args.bmname is not None:
          name = args.bmname
      else:
          name = args.dataset
      name += "_" + args.method

      name += "_h" + str(args.hidden_dim) + "_o" + str(args.output_dim)
      if not args.bias:
          name += "_nobias"
      if len(args.name_suffix) > 0:
          name += "_" + args.name_suffix
      return name


  def gen_explainer_prefix(args):
      '''Generate label prefix for a graph explainer model.
      '''
      name = gen_prefix(args) + "_explain"
      if len(args.explainer_suffix) > 0:
          name += "_" + args.explainer_suffix
      return name


  def create_filename(save_dir, args, isbest=False, num_epochs=-1):
      """
      Args:
          args        :  the arguments parsed in the parser
          isbest      :  whether the saved model is the best-performing one
          num_epochs  :  epoch number of the model (when isbest=False)
      """
      filename = os.path.join(save_dir, gen_prefix(args))
      os.makedirs(filename, exist_ok=True)

      if isbest:
          filename = os.path.join(filename, "best")
      elif num_epochs > 0:
          filename = os.path.join(filename, str(num_epochs))

      return filename + ".pth.tar"


  def save_checkpoint(model, optimizer, args, num_epochs=-1, isbest=False, cg_dict=None):
      """Save pytorch model checkpoint.
      Args:
          - model         : The PyTorch model to save.
          - optimizer     : The optimizer used to train the model.
          - args          : A dict of meta-data about the model.
          - num_epochs    : Number of training epochs.
          - isbest        : True if the model has the highest accuracy so far.
          - cg_dict       : A dictionary of the sampled computation graphs.
      """
      filename = create_filename(args.ckptdir, args, isbest, num_epochs=num_epochs)
      torch.save(
          {
              "epoch": num_epochs,
              "model_type": args.method,
              "optimizer": optimizer,
              "model_state": model.state_dict(),
              "optimizer_state": optimizer.state_dict(),
              "cg": cg_dict,
          },
          filename,
      )


  def load_ckpt(args, isbest=False):
      '''Load a pre-trained pytorch model from checkpoint.
      '''
      print("loading model")
      filename = create_filename(args.ckptdir, args, isbest)
      print(filename)
      if os.path.isfile(filename):
          print("=> loading checkpoint '{}'".format(filename))
          ckpt = torch.load(filename)
      else:
          print("Checkpoint does not exist!")
          print("Checked path -- {}".format(filename))
          print("Make sure you have provided the correct path!")
          print("You may have forgotten to train a model for this dataset.")
          print()
          print("To train one of the paper's models, run the following")
          print(">> python train.py --dataset=DATASET_NAME")
          print()
          raise Exception("File not found.")
      return ckpt

  def preprocess_cg(cg):
      """Pre-process computation graph."""
      if use_cuda:
          preprocessed_cg_tensor = torch.from_numpy(cg).cuda()
      else:
          preprocessed_cg_tensor = torch.from_numpy(cg)

      preprocessed_cg_tensor.unsqueeze_(0)
      return Variable(preprocessed_cg_tensor, requires_grad=False)

  def load_model(path):
      """Load a pytorch model."""
      model = torch.load(path)
      model.eval()
      if use_cuda:
          model.cuda()

      for p in model.features.parameters():
          p.requires_grad = False
      for p in model.classifier.parameters():
          p.requires_grad = False

      return model


  def load_cg(path):
      """Load a computation graph."""
      cg = pickle.load(open(path))
      return cg


  def save(mask_cg):
      """Save a rendering of the computation graph mask."""
      mask = mask_cg.cpu().data.numpy()[0]
      mask = np.transpose(mask, (1, 2, 0))

      mask = (mask - np.min(mask)) / np.max(mask)
      mask = 1 - mask

      cv2.imwrite("mask.png", np.uint8(255 * mask))

  def log_matrix(writer, mat, name, epoch, fig_size=(8, 6), dpi=200):
      """Save an image of a matrix to disk.
      Args:
          - writer    :  A file writer.
          - mat       :  The matrix to write.
          - name      :  Name of the file to save.
          - epoch     :  Epoch number.
          - fig_size  :  Size to of the figure to save.
          - dpi       :  Resolution.
      """
      plt.switch_backend("agg")
      fig = plt.figure(figsize=fig_size, dpi=dpi)
      mat = mat.cpu().detach().numpy()
      if mat.ndim == 1:
          mat = mat[:, np.newaxis]
      plt.imshow(mat, cmap=plt.get_cmap("BuPu"))
      cbar = plt.colorbar()
      cbar.solids.set_edgecolor("face")

      plt.tight_layout()
      fig.canvas.draw()
      writer.add_image(name, tensorboardX.utils.figure_to_image(fig), epoch)


  def denoise_graph(adj, node_idx, feat=None, label=None, threshold=None, threshold_num=None, max_component=True):
      """Cleaning a graph by thresholding its node values.
      Args:
          - adj               :  Adjacency matrix.
          - node_idx          :  Index of node to highlight (TODO ?)
          - feat              :  An array of node features.
          - label             :  A list of node labels.
          - threshold         :  The weight threshold.
          - theshold_num      :  The maximum number of nodes to threshold.
          - max_component     :  TODO
      """
      num_nodes = adj.shape[-1]
      G = nx.Graph()
      G.add_nodes_from(range(num_nodes))
      G.nodes[node_idx]["self"] = 1
      if feat is not None:
          for node in G.nodes():
              G.nodes[node]["feat"] = feat[node]
      if label is not None:
          for node in G.nodes():
              G.nodes[node]["label"] = label[node]

      if threshold_num is not None:
          # this is for symmetric graphs: edges are repeated twice in adj
          adj_threshold_num = threshold_num * 2
          #adj += np.random.rand(adj.shape[0], adj.shape[1]) * 1e-4
          neigh_size = len(adj[adj > 0])
          threshold_num = min(neigh_size, adj_threshold_num)
          threshold = np.sort(adj[adj > 0])[-threshold_num]

      if threshold is not None:
          weighted_edge_list = [
              (i, j, adj[i, j])
              for i in range(num_nodes)
              for j in range(num_nodes)
              if adj[i, j] >= threshold
          ]
      else:
          weighted_edge_list = [
              (i, j, adj[i, j])
              for i in range(num_nodes)
              for j in range(num_nodes)
              if adj[i, j] > 1e-6
          ]
      G.add_weighted_edges_from(weighted_edge_list)
      if max_component:
          largest_cc = max(nx.connected_components(G), key=len)
          G = G.subgraph(largest_cc).copy()
      else:
          # remove zero degree nodes
          G.remove_nodes_from(list(nx.isolates(G)))
      return G

  # TODO: unify log_graph and log_graph2
  def log_graph(
      writer,
      Gc,
      name,
      identify_self=True,
      nodecolor="label",
      epoch=0,
      fig_size=(4, 3),
      dpi=300,
      label_node_feat=False,
      edge_vmax=None,
      args=None,
  ):
      """
      Args:
          nodecolor: the color of node, can be determined by 'label', or 'feat'. For feat, it needs to
              be one-hot'
      """
      cmap = plt.get_cmap("Set1")
      plt.switch_backend("agg")
      fig = plt.figure(figsize=fig_size, dpi=dpi)

      node_colors = []
      # edge_colors = [min(max(w, 0.0), 1.0) for (u,v,w) in Gc.edges.data('weight', default=1)]
      edge_colors = [w for (u, v, w) in Gc.edges.data("weight", default=1)]

      # maximum value for node color
      vmax = 8
      for i in Gc.nodes():
          if nodecolor == "feat" and "feat" in Gc.nodes[i]:
              num_classes = Gc.nodes[i]["feat"].size()[0]
              if num_classes >= 10:
                  cmap = plt.get_cmap("tab20")
                  vmax = 19
              elif num_classes >= 8:
                  cmap = plt.get_cmap("tab10")
                  vmax = 9
              break

      feat_labels = {}
      for i in Gc.nodes():
          if identify_self and "self" in Gc.nodes[i]:
              node_colors.append(0)
          elif nodecolor == "label" and "label" in Gc.nodes[i]:
              node_colors.append(Gc.nodes[i]["label"] + 1)
          elif nodecolor == "feat" and "feat" in Gc.nodes[i]:
              # print(Gc.nodes[i]['feat'])
              feat = Gc.nodes[i]["feat"].detach().numpy()
              # idx with pos val in 1D array
              feat_class = 0
              for j in range(len(feat)):
                  if feat[j] == 1:
                      feat_class = j
                      break
              node_colors.append(feat_class)
              feat_labels[i] = feat_class
          else:
              node_colors.append(1)
      if not label_node_feat:
          feat_labels = None

      plt.switch_backend("agg")
      fig = plt.figure(figsize=fig_size, dpi=dpi)

      if Gc.number_of_nodes() == 0:
          raise Exception("empty graph")
      if Gc.number_of_edges() == 0:
          raise Exception("empty edge")
      # remove_nodes = []
      # for u in Gc.nodes():
      #    if Gc
      pos_layout = nx.kamada_kawai_layout(Gc, weight=None)
      # pos_layout = nx.spring_layout(Gc, weight=None)

      weights = [d for (u, v, d) in Gc.edges(data="weight", default=1)]
      if edge_vmax is None:
          edge_vmax = statistics.median_high(
              [d for (u, v, d) in Gc.edges(data="weight", default=1)]
          )
      min_color = min([d for (u, v, d) in Gc.edges(data="weight", default=1)])
      # color range: gray to black
      edge_vmin = 2 * min_color - edge_vmax
      nx.draw(
          Gc,
          pos=pos_layout,
          with_labels=False,
          font_size=4,
          labels=feat_labels,
          node_color=node_colors,
          vmin=0,
          vmax=vmax,
          cmap=cmap,
          edge_color=edge_colors,
          edge_cmap=plt.get_cmap("Greys"),
          edge_vmin=edge_vmin,
          edge_vmax=edge_vmax,
          width=1.0,
          node_size=50,
          alpha=0.8,
      )
      fig.axes[0].xaxis.set_visible(False)
      fig.canvas.draw()

      logdir = "log" if not hasattr(args, "logdir") or not args.logdir else str(args.logdir)
      if nodecolor != "feat":
          name += gen_explainer_prefix(args)
      save_path = os.path.join(logdir, name  + "_" + str(epoch) + ".pdf")
      print(logdir + "/" + name + gen_explainer_prefix(args) + "_" + str(epoch) + ".pdf")
      os.makedirs(os.path.dirname(save_path), exist_ok=True)
      plt.savefig(save_path, format="pdf")

      img = tensorboardX.utils.figure_to_image(fig)
      writer.add_image(name, img, epoch)


  def plot_cmap(cmap, ncolor):
      """
      A convenient function to plot colors of a matplotlib cmap
      Credit goes to http://gvallver.perso.univ-pau.fr/?p=712
      Args:
          ncolor (int): number of color to show
          cmap: a cmap object or a matplotlib color name
      """

      if isinstance(cmap, str):
          name = cmap
          try:
              cm = plt.get_cmap(cmap)
          except ValueError:
              print("WARNINGS :", cmap, " is not a known colormap")
              cm = plt.cm.gray
      else:
          cm = cmap
          name = cm.name

      with matplotlib.rc_context(matplotlib.rcParamsDefault):
          fig = plt.figure(figsize=(12, 1), frameon=False)
          ax = fig.add_subplot(111)
          ax.pcolor(np.linspace(1, ncolor, ncolor).reshape(1, ncolor), cmap=cm)
          ax.set_title(name)
          xt = ax.set_xticks([])
          yt = ax.set_yticks([])
      return fig


  def plot_cmap_tb(writer, cmap, ncolor, name):
      """Plot the color map used for plot."""
      fig = plot_cmap(cmap, ncolor)
      img = tensorboardX.utils.figure_to_image(fig)
      writer.add_image(name, img, 0)


  def sparse_mx_to_torch_sparse_tensor(sparse_mx):
      """Convert a scipy sparse matrix to a torch sparse tensor."""
      sparse_mx = sparse_mx.tocoo().astype(np.float32)
      indices = torch.from_numpy(
          np.vstack((sparse_mx.row, sparse_mx.col)).astype(int64)
      )
      values = torch.from_numpy(sparse_mx.data)
      shape = torch.Size(sparse_mx.shape)
      return torch.sparse.FloatTensor(indices, values, shape)

  def numpy_to_torch(img, requires_grad=True):
      if len(img.shape) < 3:
          output = np.float32([img])
      else:
          output = np.transpose(img, (2, 0, 1))

      output = torch.from_numpy(output)
      if use_cuda:
          output = output.cuda()

      output.unsqueeze_(0)
      v = Variable(output, requires_grad=requires_grad)
      return v


  def read_graphfile(datadir, dataname, max_nodes=None, edge_labels=False):
      """ Read data from https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets
          graph index starts with 1 in file
      Returns:
          List of networkx objects with graph and node labels
      """
      prefix = os.path.join(datadir, dataname, dataname)
      filename_graph_indic = prefix + "_graph_indicator.txt"
      # index of graphs that a given node belongs to
      graph_indic = {}
      with open(filename_graph_indic) as f:
          i = 1
          for line in f:
              line = line.strip("\n")
              graph_indic[i] = int(line)
              i += 1

      filename_nodes = prefix + "_node_labels.txt"
      node_labels = []
      min_label_val = None
      try:
          with open(filename_nodes) as f:
              for line in f:
                  line = line.strip("\n")
                  l = int(line)
                  node_labels += [l]
                  if min_label_val is None or min_label_val > l:
                      min_label_val = l
          # assume that node labels are consecutive
          num_unique_node_labels = max(node_labels) - min_label_val + 1
          node_labels = [l - min_label_val for l in node_labels]
      except IOError:
          print("No node labels")

      filename_node_attrs = prefix + "_node_attributes.txt"
      node_attrs = []
      try:
          with open(filename_node_attrs) as f:
              for line in f:
                  line = line.strip("\s\n")
                  attrs = [
                      float(attr) for attr in re.split("[,\s]+", line) if not attr == ""
                  ]
                  node_attrs.append(np.array(attrs))
      except IOError:
          print("No node attributes")

      label_has_zero = False
      filename_graphs = prefix + "_graph_labels.txt"
      graph_labels = []

      label_vals = []
      with open(filename_graphs) as f:
          for line in f:
              line = line.strip("\n")
              val = int(line)
              if val not in label_vals:
                  label_vals.append(val)
              graph_labels.append(val)

      label_map_to_int = {val: i for i, val in enumerate(label_vals)}
      graph_labels = np.array([label_map_to_int[l] for l in graph_labels])

      if edge_labels:
          # For Tox21_AHR we want to know edge labels
          filename_edges = prefix + "_edge_labels.txt"
          edge_labels = []

          edge_label_vals = []
          with open(filename_edges) as f:
              for line in f:
                  line = line.strip("\n")
                  val = int(line)
                  if val not in edge_label_vals:
                      edge_label_vals.append(val)
                  edge_labels.append(val)

          edge_label_map_to_int = {val: i for i, val in enumerate(edge_label_vals)}

      filename_adj = prefix + "_A.txt"
      adj_list = {i: [] for i in range(1, len(graph_labels) + 1)}
      # edge_label_list={i:[] for i in range(1,len(graph_labels)+1)}
      index_graph = {i: [] for i in range(1, len(graph_labels) + 1)}
      num_edges = 0
      with open(filename_adj) as f:
          for line in f:
              line = line.strip("\n").split(",")
              e0, e1 = (int(line[0].strip(" ")), int(line[1].strip(" ")))
              adj_list[graph_indic[e0]].append((e0, e1))
              index_graph[graph_indic[e0]] += [e0, e1]
              # edge_label_list[graph_indic[e0]].append(edge_labels[num_edges])
              num_edges += 1
      for k in index_graph.keys():
          index_graph[k] = [u - 1 for u in set(index_graph[k])]

      graphs = []
      for i in range(1, 1 + len(adj_list)):
          # indexed from 1 here
          G = nx.from_edgelist(adj_list[i])

          if max_nodes is not None and G.number_of_nodes() > max_nodes:
              continue

          # add features and labels
          G.graph["label"] = graph_labels[i - 1]

          # Special label for aromaticity experiment
          # aromatic_edge = 2
          # G.graph['aromatic'] = aromatic_edge in edge_label_list[i]

          for u in G.nodes():
              if len(node_labels) > 0:
                  node_label_one_hot = [0] * num_unique_node_labels
                  node_label = node_labels[u - 1]
                  node_label_one_hot[node_label] = 1
                  G.nodes[u]["label"] = node_label_one_hot
              if len(node_attrs) > 0:
                  G.nodes[u]["feat"] = node_attrs[u - 1]
          if len(node_attrs) > 0:
              G.graph["feat_dim"] = node_attrs[0].shape[0]

          # relabeling
          mapping = {}
          it = 0
          if float(nx.__version__) < 2.0:
              for n in G.nodes():
                  mapping[n] = it
                  it += 1
          else:
              for n in G.nodes:
                  mapping[n] = it
                  it += 1

          # indexed from 0
          graphs.append(nx.relabel_nodes(G, mapping))
      return graphs


  def read_biosnap(datadir, edgelist_file, label_file, feat_file=None, concat=True):
      """ Read data from BioSnap
      Returns:
          List of networkx objects with graph and node labels
      """
      G = nx.Graph()
      delimiter = "\t" if "tsv" in edgelist_file else ","
      print(delimiter)
      df = pd.read_csv(
          os.path.join(datadir, edgelist_file), delimiter=delimiter, header=None
      )
      data = list(map(tuple, df.values.tolist()))
      G.add_edges_from(data)
      print("Total nodes: ", G.number_of_nodes())

      G = max(nx.connected_component_subgraphs(G), key=len)
      print("Total nodes in largest connected component: ", G.number_of_nodes())

      df = pd.read_csv(os.path.join(datadir, label_file), delimiter="\t", usecols=[0, 1])
      data = list(map(tuple, df.values.tolist()))

      missing_node = 0
      for line in data:
          if int(line[0]) not in G:
              missing_node += 1
          else:
              G.nodes[int(line[0])]["label"] = int(line[1] == "Essential")

      print("missing node: ", missing_node)

      missing_label = 0
      remove_nodes = []
      for u in G.nodes():
          if "label" not in G.nodes[u]:
              missing_label += 1
              remove_nodes.append(u)
      G.remove_nodes_from(remove_nodes)
      print("missing_label: ", missing_label)

      if feat_file is None:
          feature_generator = featgen.ConstFeatureGen(np.ones(10, dtype=float))
          feature_generator.gen_node_features(G)
      else:
          df = pd.read_csv(os.path.join(datadir, feat_file), delimiter=",")
          data = np.array(df.values)
          print("Feat shape: ", data.shape)

          for row in data:
              if int(row[0]) in G:
                  if concat:
                      node = int(row[0])
                      onehot = np.zeros(10)
                      onehot[min(G.degree[node], 10) - 1] = 1.0
                      G.nodes[node]["feat"] = np.hstack(
                          (np.log(row[1:] + 0.1), [1.0], onehot)
                      )
                  else:
                      G.nodes[int(row[0])]["feat"] = np.log(row[1:] + 0.1)

          missing_feat = 0
          remove_nodes = []
          for u in G.nodes():
              if "feat" not in G.nodes[u]:
                  missing_feat += 1
                  remove_nodes.append(u)
          G.remove_nodes_from(remove_nodes)
          print("missing feat: ", missing_feat)

      return G


  def build_aromaticity_dataset():
      filename = "data/tox21_10k_data_all.sdf"
      basename = filename.split(".")[0]
      collector = []
      sdprovider = Chem.SDMolSupplier(filename)
      for i,mol in enumerate(sdprovider):
          try:
              moldict = {}
              moldict['smiles'] = Chem.MolToSmiles(mol)
              #Parse Data
              for propname in mol.GetPropNames():
                  moldict[propname] = mol.GetProp(propname)
              nb_bonds = len(mol.GetBonds())
              is_aromatic = False; aromatic_bonds = []
              for j in range(nb_bonds):
                  if mol.GetBondWithIdx(j).GetIsAromatic():
                      aromatic_bonds.append(j)
                      is_aromatic = True
              moldict['aromaticity'] = is_aromatic
              moldict['aromatic_bonds'] = aromatic_bonds
              collector.append(moldict)
          except:
              print("Molecule %s failed"%i)
      data = pd.DataFrame(collector)
      data.to_csv(basename + '_pandas.csv')


  def gen_train_plt_name(args):
      return "results/" + gen_prefix(args) + ".png"


  def log_assignment(assign_tensor, writer, epoch, batch_idx):
      plt.switch_backend("agg")
      fig = plt.figure(figsize=(8, 6), dpi=300)

      # has to be smaller than args.batch_size
      for i in range(len(batch_idx)):
          plt.subplot(2, 2, i + 1)
          plt.imshow(
              assign_tensor.cpu().data.numpy()[batch_idx[i]], cmap=plt.get_cmap("BuPu")
          )
          cbar = plt.colorbar()
          cbar.solids.set_edgecolor("face")
      plt.tight_layout()
      fig.canvas.draw()

      data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep="")
      data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      writer.add_image("assignment", data, epoch)

  # TODO: unify log_graph and log_graph2
  def log_graph2(adj, batch_num_nodes, writer, epoch, batch_idx, assign_tensor=None):
      plt.switch_backend("agg")
      fig = plt.figure(figsize=(8, 6), dpi=300)

      for i in range(len(batch_idx)):
          ax = plt.subplot(2, 2, i + 1)
          num_nodes = batch_num_nodes[batch_idx[i]]
          adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()
          G = nx.from_numpy_matrix(adj_matrix)
          nx.draw(
              G,
              pos=nx.spring_layout(G),
              with_labels=True,
              node_color="#336699",
              edge_color="grey",
              width=0.5,
              node_size=300,
              alpha=0.7,
          )
          ax.xaxis.set_visible(False)

      plt.tight_layout()
      fig.canvas.draw()

      data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep="")
      data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      writer.add_image("graphs", data, epoch)

      # log a label-less version
      # fig = plt.figure(figsize=(8,6), dpi=300)
      # for i in range(len(batch_idx)):
      #    ax = plt.subplot(2, 2, i+1)
      #    num_nodes = batch_num_nodes[batch_idx[i]]
      #    adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()
      #    G = nx.from_numpy_matrix(adj_matrix)
      #    nx.draw(G, pos=nx.spring_layout(G), with_labels=False, node_color='#336699',
      #            edge_color='grey', width=0.5, node_size=25,
      #            alpha=0.8)

      # plt.tight_layout()
      # fig.canvas.draw()

      # data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
      # data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      # writer.add_image('graphs_no_label', data, epoch)

      # colored according to assignment
      assignment = assign_tensor.cpu().data.numpy()
      fig = plt.figure(figsize=(8, 6), dpi=300)

      num_clusters = assignment.shape[2]
      all_colors = np.array(range(num_clusters))

      for i in range(len(batch_idx)):
          ax = plt.subplot(2, 2, i + 1)
          num_nodes = batch_num_nodes[batch_idx[i]]
          adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()

          label = np.argmax(assignment[batch_idx[i]], axis=1).astype(int)
          label = label[: batch_num_nodes[batch_idx[i]]]
          node_colors = all_colors[label]

          G = nx.from_numpy_matrix(adj_matrix)
          nx.draw(
              G,
              pos=nx.spring_layout(G),
              with_labels=False,
              node_color=node_colors,
              edge_color="grey",
              width=0.4,
              node_size=50,
              cmap=plt.get_cmap("Set1"),
              vmin=0,
              vmax=num_clusters - 1,
              alpha=0.8,
          )

      plt.tight_layout()
      fig.canvas.draw()

      data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep="")
      data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
      writer.add_image("graphs_colored", data, epoch)

In [ ]:
"""
  graph_utils.py
  Utility for sampling graphs from a dataset.
"""
import torch.utils.data

class graph_utils:
    class GraphSampler(torch.utils.data.Dataset):
        """ Sample graphs and nodes in graph
        """

        def __init__(
            self,
            G_list,
            features="default",
            normalize=True,
            assign_feat="default",
            max_num_nodes=0,
        ):
            self.adj_all = []
            self.len_all = []
            self.feature_all = []
            self.label_all = []

            self.assign_feat_all = []

            if max_num_nodes == 0:
                self.max_num_nodes = max([G.number_of_nodes() for G in G_list])
            else:
                self.max_num_nodes = max_num_nodes

            existing_node = list(G_list[0].nodes())[-1]
            self.feat_dim = G_list[0].nodes[existing_node]["feat"].shape[0]

            for G in G_list:
                adj = np.array(nx.to_numpy_matrix(G))
                if normalize:
                    sqrt_deg = np.diag(
                        1.0 / np.sqrt(np.sum(adj, axis=0, dtype=float).squeeze())
                    )
                    adj = np.matmul(np.matmul(sqrt_deg, adj), sqrt_deg)
                self.adj_all.append(adj)
                self.len_all.append(G.number_of_nodes())
                self.label_all.append(G.graph["label"])
                # feat matrix: max_num_nodes x feat_dim
                if features == "default":
                    f = np.zeros((self.max_num_nodes, self.feat_dim), dtype=float)
                    for i, u in enumerate(G.nodes()):
                        f[i, :] = G.nodes[u]["feat"]
                    self.feature_all.append(f)
                elif features == "id":
                    self.feature_all.append(np.identity(self.max_num_nodes))
                elif features == "deg-num":
                    degs = np.sum(np.array(adj), 1)
                    degs = np.expand_dims(
                        np.pad(degs, [0, self.max_num_nodes - G.number_of_nodes()], 0),
                        axis=1,
                    )
                    self.feature_all.append(degs)
                elif features == "deg":
                    self.max_deg = 10
                    degs = np.sum(np.array(adj), 1).astype(int)
                    degs[degs > self.max_deg] = self.max_deg
                    feat = np.zeros((len(degs), self.max_deg + 1))
                    feat[np.arange(len(degs)), degs] = 1
                    feat = np.pad(
                        feat,
                        ((0, self.max_num_nodes - G.number_of_nodes()), (0, 0)),
                        "constant",
                        constant_values=0,
                    )

                    f = np.zeros((self.max_num_nodes, self.feat_dim), dtype=float)
                    for i, u in enumerate(G.nodes()):
                        f[i, :] = G.nodes[u]["feat"]

                    feat = np.concatenate((feat, f), axis=1)

                    self.feature_all.append(feat)
                elif features == "struct":
                    self.max_deg = 10
                    degs = np.sum(np.array(adj), 1).astype(int)
                    degs[degs > 10] = 10
                    feat = np.zeros((len(degs), self.max_deg + 1))
                    feat[np.arange(len(degs)), degs] = 1
                    degs = np.pad(
                        feat,
                        ((0, self.max_num_nodes - G.number_of_nodes()), (0, 0)),
                        "constant",
                        constant_values=0,
                    )

                    clusterings = np.array(list(nx.clustering(G).values()))
                    clusterings = np.expand_dims(
                        np.pad(
                            clusterings,
                            [0, self.max_num_nodes - G.number_of_nodes()],
                            "constant",
                        ),
                        axis=1,
                    )
                    g_feat = np.hstack([degs, clusterings])
                    if "feat" in G.nodes[0]:
                        node_feats = np.array(
                            [G.nodes[i]["feat"] for i in range(G.number_of_nodes())]
                        )
                        node_feats = np.pad(
                            node_feats,
                            ((0, self.max_num_nodes - G.number_of_nodes()), (0, 0)),
                            "constant",
                        )
                        g_feat = np.hstack([g_feat, node_feats])

                    self.feature_all.append(g_feat)

                if assign_feat == "id":
                    self.assign_feat_all.append(
                        np.hstack((np.identity(self.max_num_nodes), self.feature_all[-1]))
                    )
                else:
                    self.assign_feat_all.append(self.feature_all[-1])

            self.feat_dim = self.feature_all[0].shape[1]
            self.assign_feat_dim = self.assign_feat_all[0].shape[1]

        def __len__(self):
            return len(self.adj_all)

        def __getitem__(self, idx):
            adj = self.adj_all[idx]
            num_nodes = adj.shape[0]
            adj_padded = np.zeros((self.max_num_nodes, self.max_num_nodes))
            adj_padded[:num_nodes, :num_nodes] = adj

            # use all nodes for aggregation (baseline)
            return {
                "adj": adj_padded,
                "feats": self.feature_all[idx].copy(),
                "label": self.label_all[idx],
                "num_nodes": num_nodes,
                "assign_feats": self.assign_feat_all[idx].copy(),
            }

    def neighborhoods(adj, n_hops, use_cuda):
        """Returns the n_hops degree adjacency matrix adj."""
        adj = torch.tensor(adj, dtype=torch.float)
        if use_cuda:
            adj = adj.cuda()
        hop_adj = power_adj = adj
        for i in range(n_hops - 1):
            power_adj = power_adj @ adj
            prev_hop_adj = hop_adj
            hop_adj = hop_adj + power_adj
            hop_adj = (hop_adj > 0).float()
        return hop_adj.cpu().numpy().astype(int)

# GNN Explainer

In [ ]:
from torch_geometric.explain import Explainer, GNNExplainer

In [ ]:
class Explainer1:
    def __init__(
        self,
        model,
        adj,
        feat,
        label,
        pred,
        train_idx,
        args,
        writer=None,
        print_training=True,
        graph_mode=False,
        graph_idx=False,
    ):
        self.model = model
        self.model.eval()
        self.adj = adj.detach().cpu().numpy()
        self.feat = feat.detach().cpu().numpy()
        self.label = label.detach().cpu().numpy()
        self.pred = pred.detach().cpu().numpy()
        self.train_idx = train_idx.detach().cpu().numpy()
        self.n_hops = args.num_gc_layers
        self.graph_mode = graph_mode
        self.graph_idx = graph_idx
        self.neighborhoods = None if self.graph_mode else graph_utils.neighborhoods(adj=self.adj, n_hops=self.n_hops, use_cuda=use_cuda)
        self.args = args
        self.writer = writer
        self.print_training = print_training


    # Main method
    def explain(
        self, node_idx, graph_idx=0, graph_mode=False, unconstrained=False, model="exp"
    ):
        """Explain a single node prediction
        """
        # index of the query node in the new adj
        if graph_mode:
            node_idx_new = node_idx
            sub_adj = self.adj[graph_idx]
            sub_feat = self.feat[graph_idx, :]
            sub_label = self.label[graph_idx].detach().cpu().numpy()
            neighbors = np.asarray(range(self.adj.shape[0]))
        else:
            #print("node label: ", self.label[graph_idx][node_idx])
            print("node label: ", self.label[node_idx])

            node_idx_new, sub_adj, sub_feat, sub_label, neighbors = self.extract_neighborhood(
                node_idx, graph_idx
            )
            print("neigh graph idx: ", node_idx, node_idx_new)
            sub_label = np.expand_dims(sub_label, axis=0)

        sub_adj = np.expand_dims(sub_adj, axis=0)
        sub_feat = np.expand_dims(sub_feat, axis=0)

        adj   = torch.tensor(sub_adj, dtype=torch.float)
        x     = torch.tensor(sub_feat, requires_grad=True, dtype=torch.float)
        label = torch.tensor(sub_label, dtype=torch.long)

        if self.graph_mode:
            pred_label = np.argmax(self.pred[0][graph_idx], axis=0)
            print("Graph predicted label: ", pred_label)
        else:
            #pred_label = np.argmax(self.pred[graph_idx][neighbors], axis=1)
            pred_label = np.argmax(self.pred[neighbors], axis=1)

            print("Node predicted label: ", pred_label[node_idx_new])

        explainer = ExplainModule(
            adj=adj,
            x=x,
            model=self.model,
            label=label,
            args=self.args,
            writer=self.writer,
            graph_idx=self.graph_idx,
            graph_mode=self.graph_mode,
        )
        if self.args.gpu:
            explainer = explainer.cuda()

        self.model.eval()


        # gradient baseline
        if model == "grad":
            explainer.zero_grad()
            # pdb.set_trace()
            adj_grad = torch.abs(
                explainer.adj_feat_grad(node_idx_new, pred_label[node_idx_new])[0]
            )[graph_idx]
            masked_adj = adj_grad + adj_grad.t()
            masked_adj = nn.functional.sigmoid(masked_adj)
            masked_adj = masked_adj.cpu().detach().numpy() * sub_adj.squeeze()
        else:
            explainer.train()
            begin_time = time.time()
            for epoch in range(self.args.num_epochs):
                explainer.zero_grad()
                explainer.optimizer.zero_grad()
                ypred, adj_atts = explainer(node_idx_new, unconstrained=unconstrained)
                loss = explainer.loss(ypred, pred_label, node_idx_new, epoch)
                loss.backward()

                explainer.optimizer.step()
                if explainer.scheduler is not None:
                    explainer.scheduler.step()

                mask_density = explainer.mask_density()
                if self.print_training:
                    print(
                        "epoch: ",
                        epoch,
                        "; loss: ",
                        loss.item(),
                        "; mask density: ",
                        mask_density.item(),
                        "; pred: ",
                        ypred,
                    )
                single_subgraph_label = sub_label.squeeze()

                if self.writer is not None:
                    self.writer.add_scalar("mask/density", mask_density, epoch)
                    self.writer.add_scalar(
                        "optimization/lr",
                        explainer.optimizer.param_groups[0]["lr"],
                        epoch,
                    )
                    if epoch % 25 == 0:
                        explainer.log_mask(epoch)
                        explainer.log_masked_adj(
                            node_idx_new, epoch, label=single_subgraph_label
                        )
                        explainer.log_adj_grad(
                            node_idx_new, pred_label, epoch, label=single_subgraph_label
                        )

                    if epoch == 0:
                        if self.model.att:
                            # explain node
                            print("adj att size: ", adj_atts.size())
                            adj_att = torch.sum(adj_atts[0], dim=2)
                            # adj_att = adj_att[neighbors][:, neighbors]
                            node_adj_att = adj_att * adj.float().cuda()
                            io_utils.log_matrix(
                                self.writer, node_adj_att[0], "att/matrix", epoch
                            )
                            node_adj_att = node_adj_att[0].cpu().detach().numpy()
                            G = io_utils.denoise_graph(
                                node_adj_att,
                                node_idx_new,
                                threshold=3.8,  # threshold_num=20,
                                max_component=True,
                            )
                            io_utils.log_graph(
                                self.writer,
                                G,
                                name="att/graph",
                                identify_self=not self.graph_mode,
                                nodecolor="label",
                                edge_vmax=None,
                                args=self.args,
                            )
                if model != "exp":
                    break

            print("finished training in ", time.time() - begin_time)
            if model == "exp":
                masked_adj = (
                    explainer.masked_adj[0].cpu().detach().numpy() * sub_adj.squeeze()
                )
            else:
                adj_atts = nn.functional.sigmoid(adj_atts).squeeze()
                masked_adj = adj_atts.cpu().detach().numpy() * sub_adj.squeeze()

        fname = 'masked_adj_' + io_utils.gen_explainer_prefix(self.args) + (
                'node_idx_'+str(node_idx)+'graph_idx_'+str(self.graph_idx)+'.npy')
        with open(os.path.join(self.args.logdir, fname), 'wb') as outfile:
            np.save(outfile, np.asarray(masked_adj.copy()))
            print("Saved adjacency matrix to ", fname)
        return masked_adj


    # NODE EXPLAINER
    def explain_nodes(self, node_indices, args, graph_idx=0):
        """
        Explain nodes
        Args:
            - node_indices  :  Indices of the nodes to be explained
            - args          :  Program arguments (mainly for logging paths)
            - graph_idx     :  Index of the graph to explain the nodes from (if multiple).
        """
        masked_adjs = [
            self.explain(node_idx, graph_idx=graph_idx) for node_idx in node_indices
        ]
        ref_idx = node_indices[0]
        ref_adj = masked_adjs[0]
        curr_idx = node_indices[1]
        curr_adj = masked_adjs[1]
        new_ref_idx, _, ref_feat, _, _ = self.extract_neighborhood(ref_idx)
        new_curr_idx, _, curr_feat, _, _ = self.extract_neighborhood(curr_idx)

        G_ref = io_utils.denoise_graph(ref_adj, new_ref_idx, ref_feat, threshold=0.1)
        denoised_ref_feat = np.array(
            [G_ref.nodes[node]["feat"] for node in G_ref.nodes()]
        )
        denoised_ref_adj = nx.to_numpy_matrix(G_ref)
        # ref center node
        ref_node_idx = list(G_ref.nodes()).index(new_ref_idx)

        G_curr = io_utils.denoise_graph(
            curr_adj, new_curr_idx, curr_feat, threshold=0.1
        )
        denoised_curr_feat = np.array(
            [G_curr.nodes[node]["feat"] for node in G_curr.nodes()]
        )
        denoised_curr_adj = nx.to_numpy_matrix(G_curr)
        # curr center node
        curr_node_idx = list(G_curr.nodes()).index(new_curr_idx)

        P, aligned_adj, aligned_feat = self.align(
            denoised_ref_feat,
            denoised_ref_adj,
            ref_node_idx,
            denoised_curr_feat,
            denoised_curr_adj,
            curr_node_idx,
            args=args,
        )
        io_utils.log_matrix(self.writer, P, "align/P", 0)

        G_ref = nx.convert_node_labels_to_integers(G_ref)
        io_utils.log_graph(self.writer, G_ref, "align/ref")
        G_curr = nx.convert_node_labels_to_integers(G_curr)
        io_utils.log_graph(self.writer, G_curr, "align/before")

        P = P.cpu().detach().numpy()
        aligned_adj = aligned_adj.cpu().detach().numpy()
        aligned_feat = aligned_feat.cpu().detach().numpy()

        aligned_idx = np.argmax(P[:, curr_node_idx])
        print("aligned self: ", aligned_idx)
        G_aligned = io_utils.denoise_graph(
            aligned_adj, aligned_idx, aligned_feat, threshold=0.5
        )
        io_utils.log_graph(self.writer, G_aligned, "mask/aligned")

        # io_utils.log_graph(self.writer, aligned_adj.cpu().detach().numpy(), new_curr_idx,
        #        'align/aligned', epoch=1)

        return masked_adjs


    def explain_nodes_gnn_stats(self, node_indices, args, graph_idx=0, model="exp"):
        masked_adjs = [
            self.explain(node_idx, graph_idx=graph_idx, model=model)
            for node_idx in node_indices
        ]
        # pdb.set_trace()
        graphs = []
        feats = []
        adjs = []
        pred_all = []
        real_all = []
        for i, idx in enumerate(node_indices):
            new_idx, _, feat, _, _ = self.extract_neighborhood(idx)
            G = io_utils.denoise_graph(masked_adjs[i], new_idx, feat, threshold_num=20)
            pred, real = self.make_pred_real(masked_adjs[i], new_idx)
            pred_all.append(pred)
            real_all.append(real)
            denoised_feat = np.array([G.nodes[node]["feat"] for node in G.nodes()])
            denoised_adj = nx.to_numpy_matrix(G)
            graphs.append(G)
            feats.append(denoised_feat)
            adjs.append(denoised_adj)
            io_utils.log_graph(
                self.writer,
                G,
                "graph/{}_{}_{}".format(self.args.dataset, model, i),
                identify_self=True,
                args=self.args
            )

        pred_all = np.concatenate((pred_all), axis=0)
        real_all = np.concatenate((real_all), axis=0)

        auc_all = roc_auc_score(real_all, pred_all)
        precision, recall, thresholds = precision_recall_curve(real_all, pred_all)

        plt.switch_backend("agg")
        plt.plot(recall, precision)
        plt.savefig("log/pr/pr_" + self.args.dataset + "_" + model + ".png")

        plt.close()

        auc_all = roc_auc_score(real_all, pred_all)
        precision, recall, thresholds = precision_recall_curve(real_all, pred_all)

        plt.switch_backend("agg")
        plt.plot(recall, precision)
        plt.savefig("log/pr/pr_" + self.args.dataset + "_" + model + ".png")

        plt.close()

        with open("log/pr/auc_" + self.args.dataset + "_" + model + ".txt", "w") as f:
            f.write(
                "dataset: {}, model: {}, auc: {}\n".format(
                    self.args.dataset, "exp", str(auc_all)
                )
            )

        return masked_adjs

    # GRAPH EXPLAINER
    def explain_graphs(self, graph_indices):
        """
        Explain graphs.
        """
        masked_adjs = []

        for graph_idx in graph_indices:
            masked_adj = self.explain(node_idx=0, graph_idx=graph_idx, graph_mode=True)
            G_denoised = io_utils.denoise_graph(
                masked_adj,
                0,
                threshold_num=20,
                feat=self.feat[graph_idx],
                max_component=False,
            )
            label = self.label[graph_idx]
            io_utils.log_graph(
                self.writer,
                G_denoised,
                "graph/graphidx_{}_label={}".format(graph_idx, label),
                identify_self=False,
                nodecolor="feat",
                args=self.args
            )
            masked_adjs.append(masked_adj)

            G_orig = io_utils.denoise_graph(
                self.adj[graph_idx],
                0,
                feat=self.feat[graph_idx],
                threshold=None,
                max_component=False,
            )

            io_utils.log_graph(
                self.writer,
                G_orig,
                "graph/graphidx_{}".format(graph_idx),
                identify_self=False,
                nodecolor="feat",
                args=self.args
            )

        # plot cmap for graphs' node features
        io_utils.plot_cmap_tb(self.writer, "tab20", 20, "tab20_cmap")

        return masked_adjs

    def log_representer(self, rep_val, sim_val, alpha, graph_idx=0):
        """ visualize output of representer instances. """
        rep_val = rep_val.cpu().detach().numpy()
        sim_val = sim_val.cpu().detach().numpy()
        alpha = alpha.cpu().detach().numpy()
        sorted_rep = sorted(range(len(rep_val)), key=lambda k: rep_val[k])
        print(sorted_rep)
        topk = 5
        most_neg_idx = [sorted_rep[i] for i in range(topk)]
        most_pos_idx = [sorted_rep[-i - 1] for i in range(topk)]
        rep_idx = [most_pos_idx, most_neg_idx]

        if self.graph_mode:
            pred = np.argmax(self.pred[0][graph_idx], axis=0)
        else:
            pred = np.argmax(self.pred[graph_idx][self.train_idx], axis=1)
        print(metrics.confusion_matrix(self.label[graph_idx][self.train_idx], pred))
        plt.switch_backend("agg")
        fig = plt.figure(figsize=(5, 3), dpi=600)
        for i in range(2):
            for j in range(topk):
                idx = self.train_idx[rep_idx[i][j]]
                print(
                    "node idx: ",
                    idx,
                    "; node label: ",
                    self.label[graph_idx][idx],
                    "; pred: ",
                    pred,
                )

                idx_new, sub_adj, sub_feat, sub_label, neighbors = self.extract_neighborhood(
                    idx, graph_idx
                )
                G = nx.from_numpy_matrix(sub_adj)
                node_colors = [1 for i in range(G.number_of_nodes())]
                node_colors[idx_new] = 0
                # node_color='#336699',

                ax = plt.subplot(2, topk, i * topk + j + 1)
                nx.draw(
                    G,
                    pos=nx.spring_layout(G),
                    with_labels=True,
                    font_size=4,
                    node_color=node_colors,
                    cmap=plt.get_cmap("Set1"),
                    vmin=0,
                    vmax=8,
                    edge_vmin=0.0,
                    edge_vmax=1.0,
                    width=0.5,
                    node_size=25,
                    alpha=0.7,
                )
                ax.xaxis.set_visible(False)
        fig.canvas.draw()
        self.writer.add_image(
            "local/representer_neigh", tensorboardX.utils.figure_to_image(fig), 0
        )

    def representer(self):
        """
        experiment using representer theorem for finding supporting instances.
        https://papers.nips.cc/paper/8141-representer-point-selection-for-explaining-deep-neural-networks.pdf
        """
        self.model.train()
        self.model.zero_grad()
        adj = torch.tensor(self.adj, dtype=torch.float)
        x = torch.tensor(self.feat, requires_grad=True, dtype=torch.float)
        label = torch.tensor(self.label, dtype=torch.long)
        if self.args.gpu:
            adj, x, label = adj.cuda(), x.cuda(), label.cuda()

        preds, _ = self.model(x, adj)
        preds.retain_grad()
        self.embedding = self.model.embedding_tensor
        loss = self.model.loss(preds, label)
        loss.backward()
        self.preds_grad = preds.grad
        pred_idx = np.expand_dims(np.argmax(self.pred, axis=2), axis=2)
        pred_idx = torch.LongTensor(pred_idx)
        if self.args.gpu:
            pred_idx = pred_idx.cuda()
        self.alpha = self.preds_grad


    # Utilities
    def extract_neighborhood(self, node_idx, graph_idx=0):
        """Returns the neighborhood of a given ndoe."""
        #neighbors_adj_row = self.neighborhoods[graph_idx][node_idx, :]
        neighbors_adj_row = self.neighborhoods[node_idx, :]

        # index of the query node in the new adj
        node_idx_new = sum(neighbors_adj_row[:node_idx])
        neighbors = np.nonzero(neighbors_adj_row)[0]

        #sub_adj = self.adj[graph_idx][neighbors][:, neighbors]
        sub_adj = self.adj[neighbors][:, neighbors]

        sub_feat = self.feat[graph_idx, neighbors]

        #sub_label = self.label[graph_idx][neighbors]
        sub_label = self.label[neighbors]

        return node_idx_new, sub_adj, sub_feat, sub_label, neighbors

    def align(
        self, ref_feat, ref_adj, ref_node_idx, curr_feat, curr_adj, curr_node_idx, args
    ):
        """ Tries to find an alignment between two graphs.
        """
        ref_adj = torch.FloatTensor(ref_adj)
        curr_adj = torch.FloatTensor(curr_adj)

        ref_feat = torch.FloatTensor(ref_feat)
        curr_feat = torch.FloatTensor(curr_feat)

        P = nn.Parameter(torch.FloatTensor(ref_adj.shape[0], curr_adj.shape[0]))
        with torch.no_grad():
            nn.init.constant_(P, 1.0 / ref_adj.shape[0])
            P[ref_node_idx, :] = 0.0
            P[:, curr_node_idx] = 0.0
            P[ref_node_idx, curr_node_idx] = 1.0
        opt = torch.optim.Adam([P], lr=0.01, betas=(0.5, 0.999))
        for i in range(args.align_steps):
            opt.zero_grad()
            feat_loss = torch.norm(P @ curr_feat - ref_feat)

            aligned_adj = P @ curr_adj @ torch.transpose(P, 0, 1)
            align_loss = torch.norm(aligned_adj - ref_adj)
            loss = feat_loss + align_loss
            loss.backward()  # Calculate gradients
            self.writer.add_scalar("optimization/align_loss", loss, i)
            print("iter: ", i, "; loss: ", loss)
            opt.step()

        return P, aligned_adj, P @ curr_feat

    def make_pred_real(self, adj, start):
        # house graph
        if self.args.dataset == "syn1" or self.args.dataset == "syn2":
            # num_pred = max(G.number_of_edges(), 6)
            pred = adj[np.triu(adj) > 0]
            real = adj.copy()

            if real[start][start + 1] > 0:
                real[start][start + 1] = 10
            if real[start + 1][start + 2] > 0:
                real[start + 1][start + 2] = 10
            if real[start + 2][start + 3] > 0:
                real[start + 2][start + 3] = 10
            if real[start][start + 3] > 0:
                real[start][start + 3] = 10
            if real[start][start + 4] > 0:
                real[start][start + 4] = 10
            if real[start + 1][start + 4]:
                real[start + 1][start + 4] = 10
            real = real[np.triu(real) > 0]
            real[real != 10] = 0
            real[real == 10] = 1

        # cycle graph
        elif self.args.dataset == "syn4":
            pred = adj[np.triu(adj) > 0]
            real = adj.copy()
            # pdb.set_trace()
            if real[start][start + 1] > 0:
                real[start][start + 1] = 10
            if real[start + 1][start + 2] > 0:
                real[start + 1][start + 2] = 10
            if real[start + 2][start + 3] > 0:
                real[start + 2][start + 3] = 10
            if real[start + 3][start + 4] > 0:
                real[start + 3][start + 4] = 10
            if real[start + 4][start + 5] > 0:
                real[start + 4][start + 5] = 10
            if real[start][start + 5]:
                real[start][start + 5] = 10
            real = real[np.triu(real) > 0]
            real[real != 10] = 0
            real[real == 10] = 1

        return pred, real


class ExplainModule(nn.Module):
    def __init__(
        self,
        adj,
        x,
        model,
        label,
        args,
        graph_idx=0,
        writer=None,
        use_sigmoid=True,
        graph_mode=False,
    ):
        super(ExplainModule, self).__init__()
        self.adj = adj
        self.x = x
        self.model = model
        self.label = label
        self.graph_idx = graph_idx
        self.args = args
        self.writer = writer
        self.mask_act = args.mask_act
        self.use_sigmoid = use_sigmoid
        self.graph_mode = graph_mode

        init_strategy = "normal"
        num_nodes = adj.size()[1]
        self.mask, self.mask_bias = self.construct_edge_mask(
            num_nodes, init_strategy=init_strategy
        )

        self.feat_mask = self.construct_feat_mask(x.size(-1), init_strategy="constant")
        params = [self.mask, self.feat_mask]
        if self.mask_bias is not None:
            params.append(self.mask_bias)
        # For masking diagonal entries
        self.diag_mask = torch.ones(num_nodes, num_nodes) - torch.eye(num_nodes)
        if args.gpu:
            self.diag_mask = self.diag_mask.cuda()

        self.scheduler, self.optimizer = train_utils.build_optimizer(args, params)

        self.coeffs = {
            "size": 0.005,
            "feat_size": 1.0,
            "ent": 1.0,
            "feat_ent": 0.1,
            "grad": 0,
            "lap": 1.0,
        }

    def construct_feat_mask(self, feat_dim, init_strategy="normal"):
        mask = nn.Parameter(torch.FloatTensor(feat_dim))
        if init_strategy == "normal":
            std = 0.1
            with torch.no_grad():
                mask.normal_(1.0, std)
        elif init_strategy == "constant":
            with torch.no_grad():
                nn.init.constant_(mask, 0.0)
                # mask[0] = 2
        return mask

    def construct_edge_mask(self, num_nodes, init_strategy="normal", const_val=1.0):
        mask = nn.Parameter(torch.FloatTensor(num_nodes, num_nodes))
        if init_strategy == "normal":
            std = nn.init.calculate_gain("relu") * math.sqrt(
                2.0 / (num_nodes + num_nodes)
            )
            with torch.no_grad():
                mask.normal_(1.0, std)
                # mask.clamp_(0.0, 1.0)
        elif init_strategy == "const":
            nn.init.constant_(mask, const_val)

        if self.args.mask_bias:
            mask_bias = nn.Parameter(torch.FloatTensor(num_nodes, num_nodes))
            nn.init.constant_(mask_bias, 0.0)
        else:
            mask_bias = None

        return mask, mask_bias

    def _masked_adj(self):
        sym_mask = self.mask
        if self.mask_act == "sigmoid":
            sym_mask = torch.sigmoid(self.mask)
        elif self.mask_act == "ReLU":
            sym_mask = nn.ReLU()(self.mask)
        sym_mask = (sym_mask + sym_mask.t()) / 2
        adj = self.adj.cuda() if self.args.gpu else self.adj
        masked_adj = adj * sym_mask
        if self.args.mask_bias:
            bias = (self.mask_bias + self.mask_bias.t()) / 2
            bias = nn.ReLU6()(bias * 6) / 6
            masked_adj += (bias + bias.t()) / 2
        return masked_adj * self.diag_mask

    def mask_density(self):
        mask_sum = torch.sum(self._masked_adj()).cpu()
        adj_sum = torch.sum(self.adj)
        return mask_sum / adj_sum

    def forward(self, node_idx, unconstrained=False, mask_features=True, marginalize=False):
        x = self.x.cuda() if self.args.gpu else self.x

        if unconstrained:
            sym_mask = torch.sigmoid(self.mask) if self.use_sigmoid else self.mask
            self.masked_adj = (
                torch.unsqueeze((sym_mask + sym_mask.t()) / 2, 0) * self.diag_mask
            )
        else:
            self.masked_adj = self._masked_adj()
            if mask_features:
                feat_mask = (
                    torch.sigmoid(self.feat_mask)
                    if self.use_sigmoid
                    else self.feat_mask
                )
                if marginalize:
                    std_tensor = torch.ones_like(x, dtype=torch.float) / 2
                    mean_tensor = torch.zeros_like(x, dtype=torch.float) - x
                    z = torch.normal(mean=mean_tensor, std=std_tensor)
                    x = x + z * (1 - feat_mask)
                else:
                    x = x * feat_mask

        ypred, adj_att = self.model(x, self.masked_adj)
        if self.graph_mode:
            res = nn.Softmax(dim=0)(ypred[0])
        else:
            node_pred = ypred[self.graph_idx, node_idx, :]
            res = nn.Softmax(dim=0)(node_pred)
        return res, adj_att

    def adj_feat_grad(self, node_idx, pred_label_node):
        self.model.zero_grad()
        self.adj.requires_grad = True
        self.x.requires_grad = True
        if self.adj.grad is not None:
            self.adj.grad.zero_()
            self.x.grad.zero_()
        if self.args.gpu:
            adj = self.adj.cuda()
            x = self.x.cuda()
            label = self.label.cuda()
        else:
            x, adj = self.x, self.adj
        ypred, _ = self.model(x, adj)
        if self.graph_mode:
            logit = nn.Softmax(dim=0)(ypred[0])
        else:
            logit = nn.Softmax(dim=0)(ypred[self.graph_idx, node_idx, :])
        logit = logit[pred_label_node]
        loss = -torch.log(logit)
        loss.backward()
        return self.adj.grad, self.x.grad

    def loss(self, pred, pred_label, node_idx, epoch):
        """
        Args:
            pred: prediction made by current model
            pred_label: the label predicted by the original model.
        """
        mi_obj = False
        if mi_obj:
            pred_loss = -torch.sum(pred * torch.log(pred))
        else:
            pred_label_node = pred_label if self.graph_mode else pred_label[node_idx]
            gt_label_node = self.label if self.graph_mode else self.label[0][node_idx]
            logit = pred[gt_label_node]
            pred_loss = -torch.log(logit)
        # size
        mask = self.mask
        if self.mask_act == "sigmoid":
            mask = torch.sigmoid(self.mask)
        elif self.mask_act == "ReLU":
            mask = nn.ReLU()(self.mask)
        size_loss = self.coeffs["size"] * torch.sum(mask)

        # pre_mask_sum = torch.sum(self.feat_mask)
        feat_mask = (
            torch.sigmoid(self.feat_mask) if self.use_sigmoid else self.feat_mask
        )
        feat_size_loss = self.coeffs["feat_size"] * torch.mean(feat_mask)

        # entropy
        mask_ent = -mask * torch.log(mask) - (1 - mask) * torch.log(1 - mask)
        mask_ent_loss = self.coeffs["ent"] * torch.mean(mask_ent)

        feat_mask_ent = - feat_mask             \
                        * torch.log(feat_mask)  \
                        - (1 - feat_mask)       \
                        * torch.log(1 - feat_mask)

        feat_mask_ent_loss = self.coeffs["feat_ent"] * torch.mean(feat_mask_ent)

        # laplacian
        D = torch.diag(torch.sum(self.masked_adj[0], 0))
        m_adj = self.masked_adj if self.graph_mode else self.masked_adj[self.graph_idx]
        L = D - m_adj
        pred_label_t = torch.tensor(pred_label, dtype=torch.float)
        if self.args.gpu:
            pred_label_t = pred_label_t.cuda()
            L = L.cuda()
        if self.graph_mode:
            lap_loss = 0
        else:
            lap_loss = (self.coeffs["lap"]
                * (pred_label_t @ L @ pred_label_t)
                / self.adj.numel()
            )

        # grad
        # adj
        # adj_grad, x_grad = self.adj_feat_grad(node_idx, pred_label_node)
        # adj_grad = adj_grad[self.graph_idx]
        # x_grad = x_grad[self.graph_idx]
        # if self.args.gpu:
        #    adj_grad = adj_grad.cuda()
        # grad_loss = self.coeffs['grad'] * -torch.mean(torch.abs(adj_grad) * mask)

        # feat
        # x_grad_sum = torch.sum(x_grad, 1)
        # grad_feat_loss = self.coeffs['featgrad'] * -torch.mean(x_grad_sum * mask)

        loss = pred_loss + size_loss + lap_loss + mask_ent_loss + feat_size_loss
        if self.writer is not None:
            self.writer.add_scalar("optimization/size_loss", size_loss, epoch)
            self.writer.add_scalar("optimization/feat_size_loss", feat_size_loss, epoch)
            self.writer.add_scalar("optimization/mask_ent_loss", mask_ent_loss, epoch)
            self.writer.add_scalar(
                "optimization/feat_mask_ent_loss", mask_ent_loss, epoch
            )
            # self.writer.add_scalar('optimization/grad_loss', grad_loss, epoch)
            self.writer.add_scalar("optimization/pred_loss", pred_loss, epoch)
            self.writer.add_scalar("optimization/lap_loss", lap_loss, epoch)
            self.writer.add_scalar("optimization/overall_loss", loss, epoch)
        return loss

    def log_mask(self, epoch):
        plt.switch_backend("agg")
        fig = plt.figure(figsize=(4, 3), dpi=400)
        plt.imshow(self.mask.cpu().detach().numpy(), cmap=plt.get_cmap("BuPu"))
        cbar = plt.colorbar()
        cbar.solids.set_edgecolor("face")

        plt.tight_layout()
        fig.canvas.draw()
        self.writer.add_image(
            "mask/mask", tensorboardX.utils.figure_to_image(fig), epoch
        )

        # fig = plt.figure(figsize=(4,3), dpi=400)
        # plt.imshow(self.feat_mask.cpu().detach().numpy()[:,np.newaxis], cmap=plt.get_cmap('BuPu'))
        # cbar = plt.colorbar()
        # cbar.solids.set_edgecolor("face")

        # plt.tight_layout()
        # fig.canvas.draw()
        # self.writer.add_image('mask/feat_mask', tensorboardX.utils.figure_to_image(fig), epoch)
        io_utils.log_matrix(
            self.writer, torch.sigmoid(self.feat_mask), "mask/feat_mask", epoch
        )

        fig = plt.figure(figsize=(4, 3), dpi=400)
        # use [0] to remove the batch dim
        plt.imshow(self.masked_adj[0].cpu().detach().numpy(), cmap=plt.get_cmap("BuPu"))
        cbar = plt.colorbar()
        cbar.solids.set_edgecolor("face")

        plt.tight_layout()
        fig.canvas.draw()
        self.writer.add_image(
            "mask/adj", tensorboardX.utils.figure_to_image(fig), epoch
        )

        if self.args.mask_bias:
            fig = plt.figure(figsize=(4, 3), dpi=400)
            # use [0] to remove the batch dim
            plt.imshow(self.mask_bias.cpu().detach().numpy(), cmap=plt.get_cmap("BuPu"))
            cbar = plt.colorbar()
            cbar.solids.set_edgecolor("face")

            plt.tight_layout()
            fig.canvas.draw()
            self.writer.add_image(
                "mask/bias", tensorboardX.utils.figure_to_image(fig), epoch
            )

    def log_adj_grad(self, node_idx, pred_label, epoch, label=None):
        log_adj = False

        if self.graph_mode:
            predicted_label = pred_label
            # adj_grad, x_grad = torch.abs(self.adj_feat_grad(node_idx, predicted_label)[0])[0]
            adj_grad, x_grad = self.adj_feat_grad(node_idx, predicted_label)
            adj_grad = torch.abs(adj_grad)[0]
            x_grad = torch.sum(x_grad[0], 0, keepdim=True).t()
        else:
            predicted_label = pred_label[node_idx]
            # adj_grad = torch.abs(self.adj_feat_grad(node_idx, predicted_label)[0])[self.graph_idx]
            adj_grad, x_grad = self.adj_feat_grad(node_idx, predicted_label)
            adj_grad = torch.abs(adj_grad)[self.graph_idx]
            x_grad = x_grad[self.graph_idx][node_idx][:, np.newaxis]
            # x_grad = torch.sum(x_grad[self.graph_idx], 0, keepdim=True).t()
        adj_grad = (adj_grad + adj_grad.t()) / 2
        adj_grad = (adj_grad * self.adj).squeeze()
        if log_adj:
            io_utils.log_matrix(self.writer, adj_grad, "grad/adj_masked", epoch)
            self.adj.requires_grad = False
            io_utils.log_matrix(self.writer, self.adj.squeeze(), "grad/adj_orig", epoch)

        masked_adj = self.masked_adj[0].cpu().detach().numpy()

        # only for graph mode since many node neighborhoods for syn tasks are relatively large for
        # visualization
        if self.graph_mode:
            G = io_utils.denoise_graph(
                masked_adj, node_idx, feat=self.x[0], threshold=None, max_component=False
            )
            io_utils.log_graph(
                self.writer,
                G,
                name="grad/graph_orig",
                epoch=epoch,
                identify_self=False,
                label_node_feat=True,
                nodecolor="feat",
                edge_vmax=None,
                args=self.args,
            )
        io_utils.log_matrix(self.writer, x_grad, "grad/feat", epoch)

        adj_grad = adj_grad.detach().numpy()
        if self.graph_mode:
            print("GRAPH model")
            G = io_utils.denoise_graph(
                adj_grad,
                node_idx,
                feat=self.x[0],
                threshold=0.0003,  # threshold_num=20,
                max_component=True,
            )
            io_utils.log_graph(
                self.writer,
                G,
                name="grad/graph",
                epoch=epoch,
                identify_self=False,
                label_node_feat=True,
                nodecolor="feat",
                edge_vmax=None,
                args=self.args,
            )
        else:
            # G = io_utils.denoise_graph(adj_grad, node_idx, label=label, threshold=0.5)
            G = io_utils.denoise_graph(adj_grad, node_idx, threshold_num=12)
            io_utils.log_graph(
                self.writer, G, name="grad/graph", epoch=epoch, args=self.args
            )

        # if graph attention, also visualize att

    def log_masked_adj(self, node_idx, epoch, name="mask/graph", label=None):
        # use [0] to remove the batch dim
        masked_adj = self.masked_adj[0].cpu().detach().numpy()
        if self.graph_mode:
            G = io_utils.denoise_graph(
                masked_adj,
                node_idx,
                feat=self.x[0],
                threshold=0.2,  # threshold_num=20,
                max_component=True,
            )
            io_utils.log_graph(
                self.writer,
                G,
                name=name,
                identify_self=False,
                nodecolor="feat",
                epoch=epoch,
                label_node_feat=True,
                edge_vmax=None,
                args=self.args,
            )
        else:
            G = io_utils.denoise_graph(
                masked_adj, node_idx, threshold_num=12, max_component=True
            )
            io_utils.log_graph(
                self.writer,
                G,
                name=name,
                identify_self=True,
                nodecolor="label",
                epoch=epoch,
                edge_vmax=None,
                args=self.args,
            )

# NetworkX Plotting


In [ ]:
import matplotlib.pyplot as plt
import csrgraph as cg
import numpy as np

class NXG:
    def get_correct_idx(predictions, labels, get_correct=True):
      """
      input: tensor of predicted values of shape [num_nodes, num_labels], tensor of ground truth labels of shape [num_nodes],
              bool determining whether to get indices of correctly or incorrectly predicted nodes
      output: tensor of indices (correct or incorrect)
      """
      if get_correct:
        idx = (predictions.argmax(1) == labels)
      else:
        idx = (predictions.argmax(1) != labels)

      idx = Utils.bool_to_idx(idx).flatten()
      return idx

    def protected_node_ratio(nodes, protected_set):
      """
      input: tensor of node indices, tensor of protected indices
      output: percentage of protected nodes in nodes
      """
      protected = 0
      for i in nodes:
        if protected_set[i] == True:
          protected += 1
      return (float(protected) / len(nodes))


    def plot_wrong_pred(nx_graph, pred, labels):
      """
      input: networkx graph, tensor of predicted values of shape [num_nodes, num_labels], tensor of ground truth labels of shape [num_nodes]
      output: diagram with incorrectly predicted nodes colored
      """
      pos = nx.spring_layout(nx_graph, seed=42)

      wrong_idx = NXG.get_correct_idx(pred, labels, False).cpu()

      wrong_bin = np.where(wrong_idx == True, 1, 0.25)
      node_size = list(map(lambda x: x*100, wrong_bin))

      plt.figure(figsize=(13, 13))
      nodes = nx.draw_networkx_nodes(nx_graph, pos,
                                    cmap=plt.cm.plasma,
                                    node_color=wrong_bin,
                                    node_size=0.5,
                                    alpha=wrong_bin
                                    )

      edges = nx.draw_networkx_edges(nx_graph, pos, width=0.25, alpha=0.3)
      plt.show()

    def plot_sets(G, protected_set):
      """
      input: networkx graph and a 1-dim tensor of indices of protected set
      output: diagram with nodes in the protected set colored red and all other nodes colored blue /
                edges between two nodes of the same type will be the same color as their specified node types /
                else they will be greeen
      """
      community_map = {}
      protected_set = Utils.bool_to_idx(protected_set)
      for node in G.nodes():
        if node in protected_set:
          community_map[node] = 1
        else:
          community_map[node] = 0

      node_color = []
      color_map = {0: 0, 1: 1}
      node_color = [color_map[community_map[node]] for node in G.nodes()]

      node_type_map = {0:'gX', 1:'g0'}
      node_types = {node:node_type_map[community_map[node]] for node in G.nodes()}
      nx.set_node_attributes(G, node_types, 'node_type')

      node_label_map = {0:0, 1:1}
      node_labels = {node:node_label_map[community_map[node]] for node in G.nodes()}
      nx.set_node_attributes(G, node_labels, 'node_label')

      edge_type_map = {'00':'e0', '11':'e1', '01':'e2', '10':'e2'}
      edge_types = {}

      for edge in G.edges:
        class_node_0 = str(community_map[edge[0]])
        class_node_1 = str(community_map[edge[1]])
        edge_type = class_node_0 + class_node_1
        edge_types[edge] = edge_type_map[edge_type]

      nx.set_edge_attributes(G, edge_types, 'edge_type')

      edge_color = {}
      for edge in G.edges():
        n1, n2 = edge
        edge_color[edge] = community_map[n1] if community_map[n1] == community_map[n2] else 2
        if community_map[n1] == community_map[n2] and community_map[n1] == 0:
          edge_color[edge] = 'blue'
        elif community_map[n1] == community_map[n2] and community_map[n1] == 1:
          edge_color[edge] = 'red'
        else:
          edge_color[edge] = 'green'

      pos = nx.spring_layout(G)
      nx.classes.function.set_edge_attributes(G, edge_color, name='color')
      colors = nx.get_edge_attributes(G,'color').values()
      labels = nx.get_node_attributes(G, 'node_type')

      plt.figure(figsize=(18, 18))
      nodes = nx.draw_networkx_nodes(G, pos=pos, cmap=plt.get_cmap('coolwarm'), node_color=node_color, node_size=0.5)
      edges = nx.draw_networkx_edges(G, pos=pos, edge_color=colors, width=0.35, alpha=0.3)

      plt.show()


    def plot_edge_changes(nx_graph, locked_adj, base_adj):
      """

      """
      diff = locked_adj - base_adj
      edge_color = {}
      for edge in  nx_graph.edges:
        e1, e2 = edge
        if diff[e1][e2] == 1:
          edge_color[edge] = 'red'
        else:
          edge_color[edge] = 'blue'

      print(edge_color)
      pos = nx.spring_layout(nx_graph)
      nx.classes.function.set_edge_attributes(nx_graph, edge_color, name='color')
      colors = nx.get_edge_attributes(nx_graph,'color').values()

      plt.figure(figsize=(15, 15))
      nodes = nx.draw_networkx_nodes(nx_graph, pos=pos, cmap=plt.get_cmap('coolwarm'), node_size=0.5)
      edges = nx.draw_networkx_edges(nx_graph, pos=pos, edge_color=colors, width=0.3, alpha=0.3)

      plt.show()

    def avg_degree(nx_graph, ind):
      """
      input: networkx graph, tensor of node indices to be averaged over
      output: float value average degree of nodes in the set of indices
      """
      avg = 0
      for i in ind:
        avg += nx_graph.degree[int(i)]
      return avg / float(len(ind))


    def protected_neighbor_ratio(nx_graph, protected_set):
      """
      input: networkx graph, 1-dim tensor of protected nodes in the graph
      output: 1-dim tensor of size [num_nodes] where each node index is equal to avg protected neighbor ratio degree
      """
      ratio = torch.zeros(graph.num_nodes)
      for i in range(nx_graph.number_of_nodes()):
        protected_count = 0
        for j in nx_graph.neighbors(i):
          if j in protected_set:
            protected_count += 1
        if graph.degree[i] != 0:
          ratio[i] = protected_count / graph.degree[i]
        else:
          ratio[i] = 0
      return ratio


    def avg_clustering(nx_graph, ind):
      """
      input: networkx graph, tensor of node indices to be averaged over
      output: float value average clustering of nodes in the set of indices
      """
      sum = 0
      for i in ind:
        sum += nx.clustering(nx_graph, int(i))
      return sum / len(ind)


    def manual_clustering(nx_graph):
      """
      self-implementation of the networkx clustering method
      input: networkx graph
      output: dictionary where each key corresponds to a node and the value is the clustering coefficient of the key
      """
      result = {}

      for node in range(nx_graph.number_of_nodes()):
      # Iterate over nodes of g
        neighbors = list(nx_graph.neighbors(node))
        n_neighbors = len(neighbors)
        n_links = 0

        if n_neighbors > 1:
          for node1 in neighbors:
            for node2 in neighbors:
              if nx_graph.has_edge(node1,node2):
                n_links += 1

          n_links /= 2 # n_links is calculated twice
          result[node] = 2*n_links/(n_neighbors*(n_neighbors-1))

        else:
          result[node] = 0

      return result

    def protected_clustering_ratio(nx_graph, protected_set):
      """
      input: networkx graph, set of protected nodes
      output: tensor of size (num_nodes) where each value is the number of protected nodes per cluster at each node
      """
      protected = torch.zeros(nx_graph.number_of_nodes())

      for node in range(nx_graph.number_of_nodes()):
      # Iterate over nodes of g
        neighbors = list(nx_graph.neighbors(node))
        n_neighbors = len(neighbors)
        n_links = 0

        if n_neighbors > 1:
          for node1 in neighbors:
            for node2 in neighbors:
              if nx_graph.has_edge(node1,node2):
                n_links += 1
                protected[node] += sum(el in [node, node1, node2] for el in protected_set)

          n_links /= 2 # n_links is calculated twice
          protected[node] /= n_links

        else:
          protected[node] = 0

      return protected

    def avg_centrality(nx_graph, ind):
      """
      input: networkx graph, tensor of node indices to be averaged over
      output: float value average centrality of nodes in the set of indices
      """
      sum = 0
      for i in ind:
        sum += nx.degree_centrality(nx_graph).get(int(i))
      return (sum / len(ind))


    def label_count(indices):
      """
      input: 1-dim tensor of node indices
      output: dictionary of ground truth labels and the number of occurences of each label in indices
      """
      l = {}
      for i in range(graph.labels.max().item()+1):
        l[i] = 0
      for i in indices:
        l[int(graph.labels[i])] += 1
      return l


    def k_nearest_neighbor_ratio(nx_graph, protected_set, k=3):
      """
      input: networkx graph, 1-dim tensor of protected nodes in the graph, number of k-hops to make for a nodes neighborhood
      output: returns tensor of size [num nodes, k] with values equal to avg ratio of protected to all nodes in k-neighborhood for a given node at each value k
      """

      k_neighborhood_ratio = torch.zeros(nx_graph.number_of_nodes(), k)

      for node in range(nx_graph.number_of_nodes()):

        for i in range(1, k+1):
          k_neighbors = nx.single_source_dijkstra_path_length(nx_graph, node, cutoff=i)
          protected = int(protected_set[list(k_neighbors.keys())].sum())
          total = len(k_neighbors.keys())
          k_neighborhood_ratio[node][i-1] = float(protected) / total
      return k_neighborhood_ratio

    def avg_k_hop_ratio(idx, k, k_neighborhood_ratio):
      """
      input: set of indices, k-hop integer, tensor of size [num_nodes, k] containing k_nearest_neighbor_ratio value at each k-hop value at each node
      output: float value of avg k_neighborhood_ratio of all nodes in idx set at specified k-hop
      """
      sum = 0
      for i in idx:
        sum += k_neighborhood_ratio[i][k-1]
      return sum / len(idx)

    def avg_random_walk_protected_ratio(nx_graph, protected_set, walk_len=5, n_walks=5):
      """
      input: networkx graph, set of protected nodes, walk_len=integer value length of each walk, n_walks=integer value how many walks to do on each node
      output: tensor of shape [num_nodes] where each index is equal to the avg number of protected nodes found in all random walks per the node index
      """
      G = cg.csrgraph(nx_graph, threads=12)
      avg_random_walk_protected_ratio = torch.zeros(graph.nx_g.number_of_nodes())

      for n in G.nodes():
        walks = G.random_walks(walklen=walk_len, # length of the walks
                epochs=n_walks, # how many times to start a walk from each node
                start_nodes=[n], # the starting node. It is either a list (e.g., [2,3]) or None. If None it does it on all nodes and returns epochs*G.number_of_nodes() walks
                return_weight=1., # probability of returning to previous node, higher val -> BFS
                neighbor_weight=1.) # probability of visiting new neighbor node -> DFS
        avg = []
        for w in walks:
          protected = protected_set[w.tolist()].sum()
          total = len(w)
          avg.append(float(protected) / total)
          avg_random_walk_protected_ratio[n] = (sum(avg) / len(avg))
      return avg_random_walk_protected_ratio


    def plot_incorrect_subgraph_k_neighborhood(nx_graph, incorrect_idx, k_hops=3, with_labels=True, protected_set=None):
      """
      input: networkx graph, set of incorrectly predicted node indices, integer value of k-hops in neighborhood, with_labels-[bool] will color nodes in \
        incorrect idx according to set {g0:red, gX:blue}, protected_set is the tensor of protected nodes g0
      output: plot of incorrectly predicted nodes and their k-hop neighbors with edges in a networkx subgraph where incorrectly predicted nodes are highlighted
      """
      nodes = []
      labeled_nodes = incorrect_idx.tolist()

      for node in incorrect_idx.tolist():
        path = nx.single_source_dijkstra_path(nx_graph, source=node, cutoff=k_hops).keys()
        nodes.append(list(path))

      nodes = list(set(([item for sublist in nodes for item in sublist])))
      idx = [0.2] * (len(nodes))

      for i in range(len(nodes)):
        if nodes[i] in incorrect_idx:
          idx[i] = 1

      sub = nx_graph.subgraph(nodes)

      if with_labels == True:
        community_map = {}
        protected_set = Utils.bool_to_idx(protected_set)
        for node in sub.nodes():
          if node in labeled_nodes:
            if node in protected_set:
              community_map[node] = 0
            else:
              community_map[node] = 1
          else:
            community_map[node] = -1

        node_color = []
        color_map = {0: "red", 1: "blue", -1: "black"}
        node_color = [color_map[community_map[node]] for node in sub.nodes()]

        node_type_map = {0:'g0', 1:'gX', -1:''}
        node_types = {node:node_type_map[community_map[node]] for node in sub.nodes()}
        nx.set_node_attributes(nx_graph, node_types, 'node_type')

        node_label_map = {0:0, 1:1, -1:-1}
        node_labels = {node:node_label_map[community_map[node]] for node in sub.nodes()}
        nx.set_node_attributes(nx_graph, node_labels, 'node_label')
        labels = nx.get_node_attributes(nx_graph, 'node_type')

      pos = nx.spring_layout(sub)
      plt.figure(figsize=(13, 13))

      nodes = nx.draw_networkx_nodes(sub,
                                      pos,
                                      cmap=plt.get_cmap('coolwarm'),
                                      node_color=node_color,
                                      node_size=0.5,
                                      label=labels,
                                      alpha=idx
                                    )

      edges = nx.draw_networkx_edges(sub, pos, width=0.25, alpha=0.3)
      plt.show()


# Designate protected nodes g0


In [ ]:
g0 = torch.rand(graph.features.shape[0]) <= args.protect_size
g0 = g0.to(device)
gX = ~g0.to(device)

print(f"Number of protected nodes: {g0.sum():.0f}")
print(f"Ratio of protected nodes: {g0.sum() / graph.features.shape[0]:.2%}")

Number of protected nodes: 246
Ratio of protected nodes: 9.08%


In [ ]:
print(f"{args.dataset} : Nodes={graph.numNodes()} : g0={g0.sum()} : gx={gX.sum()} : train={graph.idx_train.sum()} : val={graph.idx_val.sum()} : test={graph.idx_test.sum()}")

cora : Nodes=2708 : g0=246 : gx=2462 : train=140 : val=500 : test=1000


# Sampling matrix

In [ ]:
samplingMatrix = SamplingMatrix(g0, gX, graph.adj, args.sample_size)
print((samplingMatrix.sampling_matrix))

first run
tensor([[0.0000e+00, 5.5015e-05, 5.5015e-05,  ..., 5.5015e-05, 5.5015e-05,
         5.5015e-05],
        [0.0000e+00, 0.0000e+00, 5.5015e-05,  ..., 5.5015e-05, 5.5015e-05,
         5.5015e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.5015e-05, 5.5015e-05,
         5.5015e-05],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 5.5015e-05,
         5.5015e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         5.5015e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


In [ ]:
samplingMatrix.get_sample()

tensor([[   2,    7,   14,  ..., 2475, 2519, 2555],
        [2365, 1077, 2178,  ..., 2657, 2597, 2640]])

In [ ]:
samplingMatrix.getRatio()

G0:  0.33
GX:  0.33
G0GX:0.33


# Surrogate Model


In [ ]:
surrogate_gcn = GCN_ADJ(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"surrogate_gcn"
).to(device)


In [ ]:
surrogate_gcn_cs = GCN_CS(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"surrogate_gcn_cs"
).to(device)

# Generate Perturbations


In [ ]:
def generate_perturbations(surrogate, graph):
  perturbations = torch.zeros_like(graph.adj).float()
  count = torch.zeros_like(graph.adj).float()
  num_perturbations = args.ptb_rate * graph.adj.sum()

  t = tqdm(range(args.ptb_epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
  t.set_description("Perturbing")

  for epoch in t:

      # Re-initialize adj_grad
      adj_grad = torch.zeros_like(graph.adj).float()

      # Get modified adj
      modified_adj = Utils.get_modified_adj(graph.adj, perturbations).float().to(device)

      if args.do_sampling == 'Y':

          for sample_epoch in range(args.num_samples):
              # Get sample indices
              # sampled = torch.bernoulli(sampling_matrix)

              idx = samplingMatrix.get_sample()

              # Map sample to adj
              sample = modified_adj[idx[0], idx[1]].clone().detach().requires_grad_(True)
              modified_adj[idx[0], idx[1]] = sample

              # Get grad
              predictions = surrogate.predict(graph.features, modified_adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)

              loss = F.cross_entropy(predictions[g0], graph.labels[g0]) \
                  - F.cross_entropy(predictions[gX], graph.labels[gX])

              grad = torch.autograd.grad(loss, sample)[0]

              if sample_epoch == 0:
                print(grad)
              # Implement averaging
              adj_grad[idx[0], idx[1]] += grad
              count[idx[0], idx[1]] += 1

              # Update the sampling matrix
              samplingMatrix.updateByGrad(adj_grad, count)
              #samplingMatrix.getRatio()

              # Average the gradient
              adj_grad = torch.div(adj_grad, count)
              adj_grad[adj_grad != adj_grad] = 0


      else:
          # Get grad
          modified_adj = modified_adj.clone().detach().requires_grad_(True).to(device)
          predictions = surrogate.predict(graph.features, modified_adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)

          loss = F.cross_entropy(predictions[g0], graph.labels[g0]) \
              - F.cross_entropy(predictions[gX], graph.labels[gX])

          adj_grad = torch.autograd.grad(loss, modified_adj)[0]

      # Update perturbations
      lr = (num_perturbations) / (epoch + 1)
      pre_projection = int(perturbations.sum() / 2)
      perturbations = perturbations + (lr * adj_grad)
      perturbations = Utils.projection(perturbations, num_perturbations)


      # Train the model
      modified_adj = Utils.get_modified_adj(graph.adj, perturbations)
      surrogate.train1epoch(graph.features, modified_adj, graph.labels, graph.idx_train, graph.idx_test)
      t.set_postfix({"adj_l": loss.item(),
                      "adj_g": int(adj_grad.sum()),
                      "pre-p": pre_projection,
                      "target": int(num_perturbations / 2),
                      "loss": loss})


  return perturbations


In [ ]:
#perturbations_gcn = generate_perturbations(surrogate_gcn, graph)

#perturbations_gcn_cs = generate_perturbations(surrogate_gcn_cs, graph)

In [ ]:
#reset parameters (gradient --)

## TODO: generate_perturbations on (surrogate_gcn_cs, graph) is taking a significantly longer time. Investigation is required.

In [ ]:
#perturbations_gcn_cs = generate_perturbations(surrogate_gcn_cs, graph)

# runs when replacing nan with 0 but best sample output is off and very slow

# Get best perturbation



In [ ]:
def get_best_perturbation(perturbations, graph, surrogate):
  with torch.no_grad():

      max_loss = -1000

      for k in range(0,3):
          sample = torch.bernoulli(perturbations)
          modified_adj = Utils.get_modified_adj(graph.adj, perturbations)
          modified_adj = Utils.make_symmetric(modified_adj) # Removing this creates "impossible" adj, but works well

          predictions = surrogate.predict(graph.features, modified_adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)

          loss = F.cross_entropy(predictions[g0], graph.labels[g0]) \
              - F.cross_entropy(predictions[gX], graph.labels[gX])

          if loss > max_loss:
              max_loss = loss
              best = sample

      print(f"Best sample loss: {max_loss:.2f}\t Edges: {best.abs().sum() / 2:.0f}")
  return best

In [ ]:
#best_gcn_perturbation = get_best_perturbation(perturbations_gcn, graph, surrogate_gcn)

In [ ]:
#best_gcn_cs_perturbation = get_best_perturbation(perturbations_gcn_cs, graph, surrogate_gcn_cs)

# Best sample loss: 0.02	 Edges: 232

# Count Edge Changes

In [ ]:
"""
locked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)

edge_changes = abs(graph.adj - locked_adj_gcn)
num_edge_changes = edge_changes.sum()

print(f"       Dataset: {args.dataset}")
print(f"         Nodes: {graph.numNodes()}")
print(f"Original edges: {int(graph.numEdges())}")
print(f"  Edge changes: {int(num_edge_changes)}")
"""

'\nlocked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)\n\nedge_changes = abs(graph.adj - locked_adj_gcn)\nnum_edge_changes = edge_changes.sum()\n\nprint(f"       Dataset: {args.dataset}")\nprint(f"         Nodes: {graph.numNodes()}")\nprint(f"Original edges: {int(graph.numEdges())}")\nprint(f"  Edge changes: {int(num_edge_changes)}")\n'

# GNNExplainer Analysis

In [ ]:
def evaluation(model, features, adj, labels, idx_train, idx_val, idx_test, name=""):
  model.fit(features, adj, labels, idx_train, idx_test, args.reg_epochs)

  model_pred = model.predict(features, adj, labels, idx_train, idx_val, idx_test)
  model_acc = Metrics.partial_acc(model_pred, labels, g0, gX)

  print('{} Acc:'.format(name))
  print(model_acc)

  return model_pred, model_acc

In [ ]:
baseline = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline"
).to(device)
baseline_pred, baseline_acc = evaluation(baseline, graph.features, graph.edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "baseline")

Training baseline: 100%|██████████| 100/100 [00:02<00:00, 36.73it/s, loss=0.02]


G0: 79.27%
GX: 78.68%
baseline Acc:
{'g0': 0.792682945728302, 'gX': 0.7867587208747864}


In [ ]:
locked_edge_index = (locked_adj_gcn.nonzero().t().contiguous())

NameError: name 'locked_adj_gcn' is not defined

In [ ]:
locked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)
locked_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name="locked_gcn"
).to(device)
locked_gcn_pred, locked_gcn_acc = evaluation(locked_gcn, graph.features, locked_edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "locked_gcn")

In [ ]:
#diff = locked_adj_gcn - graph.adj
#Metrics.show_metrics(diff, graph.labels, g0, graph.device)

In [ ]:
explainer = Explainer(
    model=gcn,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)

In [ ]:
explanation = explainer(graph.features, graph.edge_index)



```
# This is formatted as code
```

#

# New Edge Target Distance Analysis

In [ ]:
"""
For each of the dataset
  For each private node x, if an edge (x,y) is added that is incident to the private node, find out the shortest distance from x to y in the original graph.

Tabulate the results.
(Only for original SLL)
"""

'\nFor each of the dataset\n  For each private node x, if an edge (x,y) is added that is incident to the private node, find out the shortest distance from x to y in the original graph.\n\nTabulate the results.\n(Only for original SLL)\n'

In [ ]:
def build_new_edge_distance_data():
    d = {'dataset': [], 'source':[], 'target':[], 'distance':[]}
    df = pd.DataFrame(data=d)

    datasets = ['PolBlogs','cora','CiteSeer', 'BlogCatalog', 'flickr']

    for dataset_name in datasets:
        graph = getGraph(root='.', name=dataset_name, setting='gcn', seed=args.seed, device=device)

        graph.edge_index = graph.adj.nonzero().t().contiguous()
        graph.num_nodes = graph.numNodes() #graph.features.shape[0]
        graph.num_features = graph.features.shape[1]
        graph.num_edges = graph.numEdges() #graph.edge_index.shape[1]
        graph.device = device
        graph.adj = to_sym_adj(graph.edge_index, graph.num_nodes).to(device)

        data = Data(x=graph.features, y=graph.labels, edge_index=graph.edge_index)

        nxg = nx.Graph()
        nxg = to_networkx(data, to_undirected=True)
        graph.nx_g = nxg

        graph.clustering = nx.clustering(nxg)
        graph.centrality = nx.degree_centrality(nxg)
        graph.degree = nxg.degree

        surrogate_gcn = GCN_ADJ(
            dim_input_features= graph.num_features,
            dim_output_classes= graph.labels.max().item()+1,
            num_hidden_layers=args.hidden_layers,
            device=device,
            lr=args.model_lr,
            dropout=args.dropout,
            weight_decay=args.weight_decay,
            name=f"surrogate_gcn").to(device)

        g0 = torch.rand(graph.features.shape[0]) <= args.protect_size
        g0 = g0.to(device)
        gX = ~g0.to(device)

        samplingMatrix = SamplingMatrix(g0, gX, graph.adj, args.sample_size)

        perturbations_gcn = generate_perturbations(surrogate_gcn, graph)
        best_gcn_perturbation = get_best_perturbation(perturbations_gcn, graph, surrogate_gcn)

        locked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)

        protected_indices = list(Utils.bool_to_idx(g0).flatten())

        new_edges_adj = locked_adj_gcn - graph.adj
        print(f"Shortest Distance in Original Graph:")
        for node in protected_indices:
            new_edges = list(Utils.bool_to_idx((locked_adj_gcn - graph.adj)[node] == torch.tensor(1)).flatten())
            for idx in new_edges:
                df_index = len(df)
                try:
                  shortest_dist = nx.shortest_path_length(graph.nx_g, source=int(node), target=int(idx))
                  #print(f"\t{node} : {idx} - {shortest_dist}")
                  df.loc[df_index,'dataset'] = dataset_name
                  df.loc[df_index,'source'] = int(node)
                  df.loc[df_index,'target'] = int(idx)
                  df.loc[df_index,'distance'] = int(shortest_dist)
                except:
                  #print(f"\t{node} : {idx} - {-1}")
                  df.loc[df_index,'dataset'] = dataset_name
                  df.loc[df_index,'source'] = int(node)
                  df.loc[df_index,'target'] = int(idx)
                  df.loc[df_index,'distance'] = int(-1)


    file_name = dataset_name + '_new_edge_target_distance.csv'
    df.to_csv(file_name)
    from google.colab import files
    files.download(file_name)

In [ ]:
# Data analysis

df = pd.read_csv("/content/drive/MyDrive/all_datasets_new_edge_target_distance.csv").drop(columns=['Unnamed: 0'])
graph_analysis_df = pd.read_csv('/content/drive/MyDrive/graph_analysis.csv')

In [ ]:
datasets = ['PolBlogs','cora','CiteSeer', 'BlogCatalog', 'flickr']

In [ ]:
graph_analysis_df['2-hop_pct'] = 0
graph_analysis_df['3-hop_pct'] = 0

for dataset in datasets:
    temp = df[df['dataset']==dataset]

    two_hop   = temp['distance'] <= 2
    three_hop = temp['distance'] <= 3

    print(dataset)
    print(f"Percentage nodes:")
    print(f"  2-hop neighborhood: {sum(two_hop) / len(temp)}")
    print(f"  3-hop neighborhood: {sum(three_hop) / len(temp)}\n")

    idx = graph_analysis_df[graph_analysis_df['dataset']==dataset].index

    graph_analysis_df.loc[idx, '2-hop_pct'] = sum(two_hop) / len(temp)
    graph_analysis_df.loc[idx, '3-hop_pct'] = sum(three_hop) / len(temp)

PolBlogs
Percentage nodes:
  2-hop neighborhood: 0.6120238984316654
  3-hop neighborhood: 0.8592233009708737

cora
Percentage nodes:
  2-hop neighborhood: 0.21621621621621623
  3-hop neighborhood: 0.23216659282233051

CiteSeer
Percentage nodes:
  2-hop neighborhood: 0.6795851528384279
  3-hop neighborhood: 0.6812227074235808

BlogCatalog
Percentage nodes:
  2-hop neighborhood: 0.3825284017222597
  3-hop neighborhood: 0.9927374591482077

flickr
Percentage nodes:
  2-hop neighborhood: 0.4034410311736983
  3-hop neighborhood: 0.9967066038271535



In [ ]:
graph_analysis_df.to_csv('graph_analysis.csv')

In [ ]:
connected_nodes_df    = df[df['distance']!=-1.0]
disconnected_nodes_df = df[df['distance']==-1.0]

connected_node_ratio    = len(connected_nodes_df)    / len(df)
disconnected_node_ratio = len(disconnected_nodes_df) / len(df)

In [ ]:
def shortest_path_by_dataset():
    print(f"Average Shortest Path Distance by Dataset:\n")

    avg_node_distance_by_dataset = {}

    for dataset in datasets:
        if dataset != 'all':
          dataset_df = connected_nodes_df[connected_nodes_df['dataset']==dataset]
        else:
          dataset_df = connected_nodes_df

        avg_dist   = dataset_df['distance'].sum() / len(dataset_df)
        avg_node_distance_by_dataset[dataset] = avg_dist
        print(f"{dataset}: {avg_dist}")

    return avg_node_distance_by_dataset

In [ ]:
def graph_analysis():
    d = {'dataset': [], 'avg_clustering_coef':[], 'avg_centrality':[], 'avg_degree':[]}
    graph_analysis_df = pd.DataFrame(data=d)

    for i in range(len(datasets)):
        dataset = datasets[i]
        if dataset != 'all':
            graph = getGraph(root='.', name=dataset, setting='gcn', seed=args.seed, device=device)
            graph.edge_index = graph.adj.nonzero().t().contiguous()

            data = Data(x=graph.features, y=graph.labels, edge_index=graph.edge_index)

            nxg = nx.Graph()
            nxg = to_networkx(data, to_undirected=True)

            indices = [node for node in nxg.nodes]

            clustering = nx.clustering(nxg, nodes=list(int(i) for i in indices)).values()
            centrality = nx.closeness_centrality(nxg)
            close_cent = [centrality[i] for i in indices]
            degree     = [val for (node, val) in nxg.degree(nbunch=indices)]

            print(f"Average Graph Attributes for {dataset}")
            print(f"  clustering: {sum(clustering) / len(indices)}")
            print(f"  centrality: {sum(close_cent) / len(indices)}")
            print(f"      degree: {sum(degree)     / len(indices)}\n")

            graph_analysis_df.loc[i,'dataset']             = dataset
            graph_analysis_df.loc[i,'avg_clustering_coef'] = sum(clustering) / len(indices)
            graph_analysis_df.loc[i,'avg_centrality']      = sum(close_cent) / len(indices)
            graph_analysis_df.loc[i,'avg_degree']          = sum(degree)     / len(indices)

    return graph_analysis_df

In [ ]:
avg_node_distance_by_dataset = shortest_path_by_dataset()

Average Shortest Path Distance by Dataset:

PolBlogs: 3.1930812550281575
cora: 6.862380146644106
CiteSeer: 10.256366723259763
BlogCatalog: 2.6247341391295325
flickr: 2.599852364999148


In [ ]:
all_indices = [int(i) for i in Utils.bool_to_idx(g0 + gX).flatten()]

#graph_analysis_df = graph_analysis()

In [ ]:
graph_analysis_df

,dataset,avg_clustering_coef,avg_centrality,avg_degree,avg_shortest_path_to_new_edge,2-hop_pct,3-hop_pct
0,PolBlogs,0.262652,0.250429,22.436242,3.193081,0.612024,0.859223
1,cora,0.240673,0.137476,3.898080,6.862380,0.216216,0.232167
2,CiteSeer,0.142555,0.045350,2.739130,10.256367,0.679585,0.681223
3,BlogCatalog,0.122372,0.400284,66.105851,2.624734,0.382528,0.992737
4,flickr,0.330117,0.418222,63.297162,2.599852,0.403441,0.996707


In [ ]:
"""
graph_analysis_df['avg_shortest_path_to_new_edge'] = 0

for u,v in avg_node_distance_by_dataset.items():
    idx = graph_analysis_df[graph_analysis_df['dataset']==u].index[0]
    graph_analysis_df.loc[idx,'avg_shortest_path_to_new_edge'] = v


graph_analysis_df = graph_analysis_df[:5]
graph_analysis_df.to_csv('graph_analysis.csv', index=False)
"""

# Partial Locked Graph Analysis

In [ ]:
"""
For each dataset in the original SLL paper, for x in range(10, 50, 10)
    Randomly select x% of the private nodes, and restore its 1 neighborhood edges
    Run GCN and GCN-CS on the partially corrected graph
    Tabulate the results to indicate the performance before and after the partial correction.
"""

In [ ]:
baseline_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline_gcn"
).to(device)

In [ ]:
locked_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name="locked_gcn"
).to(device)

In [ ]:
# GCN
locked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)
partial_locked_adj = torch.zeros_like(locked_adj_gcn).copy_(locked_adj_gcn)

protected_indices = list(Utils.bool_to_idx(g0).flatten())

for x in range(10, 51, 10):
  split = round(int(g0.sum()) * (x/100))
  partial_locked_adj = torch.zeros_like(locked_adj_gcn).copy_(locked_adj_gcn)

  random.shuffle(protected_indices)
  for idx in protected_indices[:split]:
      neighbors = nxg.neighbors(int(idx))
      partial_locked_adj[idx] = graph.adj[idx]
      partial_locked_adj[list(neighbors), idx] = 1

  partial_locked_edge_index = (partial_locked_adj.nonzero().t().contiguous())

  baseline_gcn_pred, baseline_gcn_acc = evaluation(baseline_gcn, graph.features, graph.edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "baseline_gcn")
  partial_locked_gcn_pred, partial_locked_gcn_acc = evaluation(locked_gcn, graph.features, partial_locked_edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "locked_gcn")

  print()
  print(f"-- GCN --")
  print(f"  {x}% Partially Corrected")
  print(f"    Baseline: {baseline_gcn_acc}")
  print(f"    Locked  : {partial_locked_gcn_acc}")
  print()

In [ ]:
stop_here

In [ ]:
baseline_gcn_cs = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline_gcn_cs"
).to(device)

In [ ]:
locked_gcn_cs = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name="locked_gcn_cs"
).to(device)

In [ ]:
perturbations_gcn_cs = generate_perturbations(surrogate_gcn_cs, graph)

best_gcn_cs_perturbation = get_best_perturbation(perturbations_gcn_cs, graph, surrogate_gcn_cs)

In [ ]:
# GCN_CS
locked_adj_gcn_cs = Utils.get_modified_adj(graph.adj, best_gcn_cs_perturbation)
partial_locked_adj = torch.zeros_like(locked_adj_gcn_cs).copy_(locked_adj_gcn_cs)

protected_indices = list(Utils.bool_to_idx(g0).flatten())

for x in range(10, 51, 10):
  split = round(int(g0.sum()) * (x/100))
  partial_locked_adj = torch.zeros_like(locked_adj_gcn_cs).copy_(locked_adj_gcn_cs)

  random.shuffle(protected_indices)
  for idx in protected_indices[:split]:
      neighbors = nxg.neighbors(int(idx))
      partial_locked_adj[idx] = graph.adj[idx]
      partial_locked_adj[list(neighbors), idx] = 1

  partial_locked_edge_index = (partial_locked_adj.nonzero().t().contiguous())

  baseline_gcn_cs_pred, baseline_gcn_cs_acc = evaluation(baseline_gcn_cs, graph.features, graph.edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "baseline_gcn_cs")
  partial_locked_gcn_cs_pred, partial_locked_gcn_cs_acc = evaluation(locked_gcn_cs, graph.features, partial_locked_edge_index, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "locked_gcn_cs")
  print()
  print(f"-- GCN_CS --")
  print(f"  {x}% Partially Corrected")
  print(f"    Baseline: {baseline_gcn_cs_acc}")
  print(f"    Locked  : {partial_locked_gcn_cs_acc}")
  print()

In [ ]:
stop here

# Cluster Graph

In [ ]:
# build cluster dictionary #

cluster_dict = {}
available_nodes = [i for i in nxg.nodes]
i = 0

while len(available_nodes) > 0:
  cluster = f"cluster{i}"
  i += 1
  node = available_nodes[0]
  node_neighborhood = list(nx.single_source_shortest_path_length(nxg, node, cutoff=2).keys())
  cluster_dict[cluster] = node_neighborhood
  available_nodes = list(set(available_nodes) - set(node_neighborhood))

# Complement Graph

In [ ]:
comp_explainer = Explainer(
    model=gcn,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)

In [ ]:
n_path = nx.single_source_shortest_path_length(nxg, 0, cutoff=2)


In [ ]:
temp = torch.zeros_like(graph.adj)
for node in nxg.nodes():
    n_path = nx.single_source_shortest_path_length(nxg, node, cutoff=2)
    temp[node, list(n_path.keys())] = 1.
    temp[node,node] = 0.
temp = temp - graph.adj

In [ ]:
def build_two_hop_comp_adj(nxg):
  """
  build 2-hop neighbor adj matrix
  """
  comp_adj = torch.zeros_like(graph.adj)
  for node in nxg.nodes():
    n_path = nx.single_source_shortest_path_length(nxg, node, cutoff=2)
    two_hop_nh = list(set(n_path.keys()) - set(list(nxg.neighbors(0))))
    comp_adj[node, two_hop_nh] = 1.
    comp_adj[node, node] = 0.
  return comp_adj

In [ ]:
comp_adj = build_two_hop_comp_adj(nxg)
comp_edge_index = comp_adj.nonzero().t().contiguous()

In [ ]:
comp_explanation = comp_explainer(graph.features, comp_edge_index)

In [ ]:

ranked_edge_weight = {}

for idx in range((comp_edge_index).shape[1]):
  edge_weight = comp_explanation.edge_mask[idx]
  pair = comp_explanation.edge_index[:,idx]
  ranked_edge_weight[pair] = edge_weight

ranked_edge_weight = dict(sorted(ranked_edge_weight.items(), key=lambda x:x[1], reverse=True))


In [ ]:
baseline_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline_gcn"
).to(device)

In [ ]:
baseline_gcn.fit(graph.features, comp_edge_index, graph.labels, graph.idx_train, graph.idx_test, args.reg_epochs)

In [ ]:
base_pred = baseline_gcn(graph.features, comp_edge_index)

In [ ]:
Metrics.partial_acc(base_pred, graph.labels, g0, gX)

In [ ]:
locked_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline_gcn"
).to(device)

In [ ]:
locked_edge_index = locked_adj_gcn.nonzero().t().contiguous()

In [ ]:
locked_gcn.fit(graph.features, locked_edge_index, graph.labels, graph.idx_train, graph.idx_test, args.reg_epochs)

In [ ]:
locked_pred = locked_gcn(graph.features, comp_edge_index)
Metrics.partial_acc(locked_pred, graph.labels, g0, gX)

# Define Weight Matrix

In [ ]:
from torch_geometric.explain import Explainer, GNNExplainer

In [ ]:
wm_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"wm_gcn"
).to(device)

In [ ]:
explainer = Explainer(
    model=wm_gcn,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)

In [ ]:
alt_adj = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)
alt_edge_index = to_edge_index(alt_adj)

In [ ]:
(graph.adj != alt_adj).sum()

In [ ]:
explanation = explainer(graph.features, graph.edge_index)

In [ ]:
weight_matrix = torch.zeros_like(graph.adj)

In [ ]:
for i in range(graph.edge_index.shape[1]):
  u = explanation.edge_index[0, i]
  v = explanation.edge_index[1, i]
  weight_matrix[u,v] = explanation.edge_mask[i]
  weight_matrix[v,u] = explanation.edge_mask[i]

# Weight Matrix Predictions

In [ ]:
# convert to sparse matrices
from scipy import sparse
from scipy.sparse import coo_matrix

scp_weight_matrix = sparse.csr_matrix(weight_matrix)
scp_alt_adj = sparse.csr_matrix(alt_adj)

In [ ]:
sp_weight_matrix = weight_matrix.to_sparse()
sp_alt_adj = alt_adj.to_sparse()

In [ ]:
alt_edge_idx = to_edge_index(alt_adj)
alt_edge_idx

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(weight_matrix, alt_adj, test_size = 0.20, random_state = 42)

In [ ]:
sp_x_train = x_train.to_sparse()
sp_y_train = y_train.to_sparse()

In [ ]:
# Random Forest Model
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
#x_under y_under = rus.fit_resample(x_train, y_train)

In [ ]:
# Neural Net
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

    def fit(self, input_features, target_adjacency_matrix, num_epochs=1000):
        for epoch in range(num_epochs):
            self.optimizer.zero_grad()

            predicted_adjacency_matrix = self(input_features)

            flat_predicted = predicted_adjacency_matrix.view(-1)
            flat_target = target_adjacency_matrix.view(-1)

            loss = self.criterion(flat_predicted, flat_target)

            loss.backward()
            self.optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

        print("Training complete!")

In [ ]:
x_train.shape[0]

In [ ]:
#gnn = GNN(x_train.shape[0], 64, y_train.shape[1])
#gnn.fit(x_train, y_train)

#gnn_pred = gnn(x_test, y_test)

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
"""
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)

rf_pred = rf.predict(x_test)
((torch.tensor(rf_pred) == 1) and (torch.tensor(rf_pred) == y_test)).sum() / (y_test == 1).sum()
"""

In [ ]:
from sklearn.metrics import accuracy_score

#print(f'Model accuracy score : {accuracy_score(y_test, y_pred):.2%}')

# Graph Property Utils

In [ ]:
def jaccard_similarity(g, h):
    intersection = len(set.intersection(set(g.edges), set(h.edges)))
    union = len(set.union(set(g.edges), set(h.edges)))
    return round(intersection/union, 3)

In [ ]:
def avg_adamic_adar_index(graph, adj, indices):
    cnt = 0
    total = 0
    for i in Utils.bool_to_idx(indices):
      for k, j in Utils.bool_to_idx(adj[i] != 0):
        total += list(nx.adamic_adar_index(graph, [[i.item(), j.item()]]))[0][2]
        cnt += 1
    return total / cnt

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra

def calc_distance_matrix(adj, indices):
    temp_graph = csr_matrix(adj)
    dist_matrix = dijkstra(csgraph=temp_graph, directed=False, indices=indices, return_predecessors=False)
    return dist_matrix

def avg_shortest_path(adj, indices):
    dist_mat = calc_distance_matrix(adj, indices)
    disconnected_nodes = []
    for i in range(len(dist_mat[0])):
      if float.is_integer(dist_mat[0][i]) == False:
        disconnected_nodes.append(i)

    total = 0
    length = dist_mat.shape[1]

    row_mask = g0.clone()
    row_mask[disconnected_nodes] = False
    for row in dist_mat[row_mask]:
        row[disconnected_nodes] = 0
        total += (float(sum(row)) / float(length-len(disconnected_nodes)))

    return (total / row_mask.sum())

In [ ]:
#temp = list(nx.katz_centrality(originalGraph))


# Graph Properties

In [ ]:
alt_edge_index = alt_adj.nonzero().t().contiguous()

In [ ]:
# Initialize Original Graph and Altered Graph as NetworkX Graphs for Analysis
from torch_geometric.data import Data

originalData = Data(x=graph.features, y=graph.labels, edge_index=graph.edge_index)
alteredData = Data(x=graph.features, y=graph.labels, edge_index=alt_edge_index)

originalGraph = to_networkx(originalData, to_undirected=True)
alteredGraph = to_networkx(alteredData, to_undirected=True)

In [ ]:
# Node Level Analysis #

In [ ]:
print("-- avg node degree --\n")
print(f"  originalGraph|")
print(f"\t Protected: {NXG.avg_degree(originalGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_degree(originalGraph, gX):.3f}\n")

print(f"  alteredGraph|")
print(f"\t Protected: {NXG.avg_degree(alteredGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_degree(alteredGraph, gX):.3f}")

In [ ]:
print("-- avg centrality --\n")
print(f"  originalGraph|")
print(f"\t Protected: {NXG.avg_centrality(originalGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_centrality(originalGraph, gX):.3f}\n")

print(f"  alteredGraph|")
print(f"\t Protected: {NXG.avg_centrality(alteredGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_centrality(alteredGraph, gX):.3f}")

In [ ]:
print("-- avg clustering --\n")
print(f"  originalGraph|")
print(f"\t Protected: {NXG.avg_clustering(originalGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_clustering(originalGraph, (originalGraph.nodes)):.3f}\n")

print(f"  alteredGraph|")
print(f"\t Protected: {NXG.avg_clustering(alteredGraph, g0):.3f}")
print(f"\tAuthorized: {NXG.avg_clustering(alteredGraph, gX):.3f}")

In [ ]:
# Link Level Analysis

In [ ]:
dist_mat = avg_shortest_path(graph.adj, originalGraph.nodes)

disconnected_nodes = []
for i in range(len(dist_mat[0])):
  if float.is_integer(dist_mat[0][i]) == False:
    disconnected_nodes.append(i)

In [ ]:
# Avg distance of protected nodes to all other nodes #
total = 0
length = dist_mat.shape[1]

row_mask = g0.clone()
row_mask[disconnected_nodes] = False

for row in dist_mat[row_mask]:
    row[disconnected_nodes] = 0
    total += (float(sum(row)) / float(length-len(disconnected_nodes)))

total = total / row_mask.sum()

In [ ]:
total

In [ ]:
print("-- avg shortest path to all other nodes --\n")
print(f"  originalGraph|")
print(f"\t Protected: {avg_shortest_path(graph.adj, g0):.4f}")
print(f"\tAuthorized: {avg_shortest_path(graph.adj, gX):.4f}\n")

print(f"  alteredGraph|")
print(f"\t Protected: {avg_shortest_path(alt_adj, g0):.4f}")
print(f"\tAuthorized: {avg_shortest_path(alt_adj, gX):.4f}")

In [ ]:
print("-- jaccard similarity --\n")
print("originalGraph vs alteredGraph")
print(jaccard_similarity(originalGraph, alteredGraph))

In [ ]:
print("-- avg adamic-adar index --\n")
print(f"  originalGraph|")
print(f"\t Protected: {avg_adamic_adar_index(originalGraph, graph.adj, g0):.4f}")
print(f"\tAuthorized: {avg_adamic_adar_index(originalGraph, graph.adj, gX):.4f}\n")

print(f"  alteredGraph|")
print(f"\t Protected: {avg_adamic_adar_index(alteredGraph, alt_adj, g0):.4f}")
print(f"\tAuthorized: {avg_adamic_adar_index(alteredGraph, alt_adj, gX):.4f}")

In [ ]:
# Graph Level Analysis

In [ ]:
"""
link level features:
    Shortest-path distance between two nodes, Common neighbors,
    Jaccard’s coefficient, Adamic-Adar index, Katz index
"""

In [ ]:
"""
graph level features: Graph Kernels
"""

# Deleted Edge Rankings

In [ ]:
diff = alt_adj - graph.adj
deleted_edge_adj_mask = (diff == -1)
print(f"Number of edges deleted: {deleted_edge_adj_mask.sum()}")

In [ ]:
# Edge removals typically occur between g0-gX of same label
Metrics.show_metrics(diff, graph.labels, g0, graph.device)

In [ ]:
deleted_edge_adj_idx = Utils.bool_to_idx(diff == -1)
# deleted_index = deleted_adj_idx.nonzero().t().contiguous()

In [ ]:
explanation.edge_index

In [ ]:
deleted_edge_adj_idx

In [ ]:
deleted_edge_mask = graph.edge_index[0] != graph.edge_index[0]

In [ ]:
for pair in deleted_edge_adj_idx:
  for i in range(graph.edge_index.shape[1]):
    if graph.edge_index[0, i] == pair[0] and graph.edge_index[1, i] == pair[1]:
      deleted_edge_mask[i] = True

In [ ]:
deleted_edge_mask.sum()

In [ ]:
deleted_edge_mask_idx = Utils.bool_to_idx(deleted_edge_mask).reshape(-1)
deleted_edge_mask_idx

In [ ]:
explanation.edge_index[:,124]

In [ ]:
num_edges_deleted_per_node = {}

for i in range(graph.num_nodes):
  if ((diff[i] == -1).sum() > 0):
    num_edges_deleted_per_node[i] = int((diff[i] == -1).sum())

In [ ]:
num_edges_deleted_per_node = dict(sorted(num_edges_deleted_per_node.items(), key=lambda x:x[1], reverse=True))

In [ ]:
num_edges_deleted_per_node

In [ ]:
# number of protected nodes that had edge removed
g0[list(num_edges_deleted_per_node.keys())].sum()

In [ ]:
deleted_edge_weight_dic = {}

for idx in deleted_edge_mask_idx:
  edge_weight = explanation.edge_mask[idx]
  pair = explanation.edge_index[:,idx]
  deleted_edge_weight_dic[pair] = edge_weight

In [ ]:
deleted_edge_weight_dic = dict(sorted(deleted_edge_weight_dic.items(), key=lambda x:x[1], reverse=True))

In [ ]:
deleted_edge_weight_dic

In [ ]:
for pair in list(deleted_edge_weight_dic.keys()):
  print(f"Edge: {pair}\tWeight: {deleted_edge_weight_dic[pair]}\tSame node type: {g0[pair[0]]==g0[pair[1]]}\tSame label: {graph.labels[pair[0]]==graph.labels[pair[1]]}")

# Evaluation



## Sample Evaluation

In [ ]:
baseline = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline"
).to(device)

baseline.fit(graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_test, args.reg_epochs)

baseline_pred = baseline.predict(graph.features, graph.adj)
baseline_acc = Metrics.partial_acc(baseline_pred, graph.labels, g0, gX)

print('Baseline Acc:')
print(baseline_acc)


In [ ]:
baseline_post_pred = baseline.post(graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
baseline_post_acc = Metrics.partial_acc(baseline_post_pred, graph.labels, g0, gX)

print('Baseline Post Acc:')
print(baseline_post_acc)

## Evaluation function

In [ ]:
def evaluation(model, features, adj, labels, idx_train, idx_val, idx_test, name=""):
  model.fit(features, adj, labels, idx_train, idx_test, args.reg_epochs)

  model_pred = model.predict(features, adj, labels, idx_train, idx_val, idx_test)
  model_acc = Metrics.partial_acc(model_pred, labels, g0, gX)

  print('{} Acc:'.format(name))
  print(model_acc)

  return model_pred, model_acc

In [ ]:
baseline = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"baseline"
).to(device)
baseline_pred, baseline_acc = evaluation(baseline, graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "baseline")

In [ ]:
baseline_post_pred = baseline.post(graph.features, graph.adj, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
baseline_post_acc = Metrics.partial_acc(baseline_post_pred, graph.labels, g0, gX)

print('baseline post Acc:')
print(baseline_post_acc)

In [ ]:
locked_adj_gcn = Utils.get_modified_adj(graph.adj, best_gcn_perturbation)
locked_gcn = GCN(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name="locked_gcn"
).to(device)
locked_gcn_pred, locked_gcn_acc = evaluation(locked_gcn, graph.features, locked_adj_gcn, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "locked_gcn")

In [ ]:
locked_gcn_post_pred = locked_gcn.post(graph.features, locked_adj_gcn, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
locked_gcn_post_acc = Metrics.partial_acc(locked_gcn_post_pred, graph.labels, g0, gX)
print('locked gcn post Acc:')
print(locked_gcn_post_acc)

In [ ]:
cs = Correct_and_Smooth(locked_adj_gcn, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
manual_locked_gcn_post_pred = cs.correct_and_smooth(locked_gcn_pred, a1=0.9, a2=0.9, eps=1e-6)
manual_locked_gcn_post_acc = Metrics.partial_acc(manual_locked_gcn_post_pred, graph.labels, g0, gX)
print('manual locked gcn post Acc:')
print(manual_locked_gcn_post_acc)

## Comment: Correct and Smooth on the predictions of locked_gcn model shows no significant improvement. It suggests that the locked model achieves its goal in protecting the protected nodes. Hence, there is no need to improve the locked model to secure againts correct and smooth.

In [ ]:
locked_adj_gcn_cs = Utils.get_modified_adj(graph.adj, best_gcn_cs_perturbation)
locked_gcn_cs = GCN_CS(
    dim_input_features= graph.num_features,
    dim_output_classes= graph.labels.max().item()+1,
    num_hidden_layers=args.hidden_layers,
    device=device,
    lr=args.model_lr,
    dropout=args.dropout,
    weight_decay=args.weight_decay,
    name=f"locked_gcn_cs"
).to(device)
locked_gcn_cs_pred, locked_gcn_cs_acc = evaluation(locked_gcn_cs, graph.features, locked_adj_gcn_cs, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test, "locked_gcn_cs")

In [ ]:
locked_gcn_cs_post_pred = locked_gcn_cs.post(graph.features, locked_adj_gcn_cs, graph.labels, graph.idx_train, graph.idx_val, graph.idx_test)
locked_gcn_cs_post_acc = Metrics.partial_acc(locked_gcn_cs_post_pred, graph.labels, g0, gX)

print('locked gcn cs post Acc:')
print(locked_gcn_cs_post_acc)

## Comment: Using surrogate locked_gcn_cs gives higher accuracy to protected nodes. This indicates that the using locked_gcn_cs model gives worst result than locked_gcn_model.

---


## TODO: What's the following for?

In [ ]:
args.reg_epochs = 100

In [ ]:
# Timer
import time

In [ ]:
base_acc = {'g0':0, 'gX':0}
locked_accu = {'g0':0, 'gX':0}

gcn_time = []

avg_dgX = 0
avg_dg0 = 0

In [ ]:
cs = Correct_and_Smooth(graph)

In [ ]:
"""
from torch_geometric.nn.models.correct_and_smooth import CorrectAndSmooth
cs = CorrectAndSmooth(num_correction_layers=5, correction_alpha=0.8,
                      num_smoothing_layers=5, smoothing_alpha=0.8)
"""

In [ ]:
"""
print(graph.labels.size(0))
numel = int(graph.idx_train.sum()) if graph.idx_train.dtype == torch.bool else graph.idx_train.size(0)
train_indices = (Utils.bool_to_idx(graph.idx_train))
"""

In [ ]:
ep = 5
for i in range(ep):
    start = time.time()

    baseline_model = GNN(
        input_features=graph.features.shape[1],
        output_classes=graph.labels.max().item()+1,
        hidden_layers=args.hidden_layers,
        device=device,
        lr=args.model_lr,
        dropout=args.dropout,
        weight_decay=args.weight_decay,
        name=f"baseline"
        ).to(device)

    baseline_model.fit(graph, args.reg_epochs)

    base_pred = baseline_model.post(graph.features, graph.adj)
    #base_pred = baseline_model(graph.features, graph.adj)
    #base_pred = cs.correct_and_smooth(base_pred, a1=0.9, a2=0.9, eps=1e-6, show_p=True)

    baseline_acc = Metrics.partial_acc(base_pred, graph.labels, g0, gX)

    print('Baseline Acc:')
    print(baseline_acc)

    locked_adj = Utils.get_modified_adj(graph.adj, best)

    locked_model = GNN(
        input_features=graph.features.shape[1],
        output_classes=graph.labels.max().item()+1,
        hidden_layers=args.hidden_layers,
        device=device,
        lr=args.model_lr,
        dropout=args.dropout,
        weight_decay=args.weight_decay,
        name=f"locked")

    locked_model.fitManual(graph.features, locked_adj, graph.labels, graph.idx_train, graph.idx_test, args.reg_epochs)
    locked_pred = locked_model.post(graph.features, locked_adj).to(device)
    #locked_pred = locked_model(graph.features, locked_adj).to(device)
    #locked_pred = cs.correct_and_smooth(locked_pred, a1=0.9, a2=0.9, eps=1e-5)

    locked_acc = Metrics.partial_acc(locked_pred, graph.labels, g0, gX)

    print('Locked Acc:')
    print(locked_acc)

    end = time.time()

    base_acc['g0'] += baseline_acc['g0']
    base_acc['gX'] += baseline_acc['gX']

    locked_accu['g0'] += locked_acc['g0']
    locked_accu['gX'] += locked_acc['gX']

    gcn_time.append(end-start)

    dg0 = (locked_acc["g0"] - baseline_acc["g0"]) / baseline_acc["g0"]
    dgX = (locked_acc["gX"] - baseline_acc["gX"]) / baseline_acc["gX"]

    avg_dgX += dgX
    avg_dg0 += dg0

In [ ]:
print(f"base_acc: gX={base_acc['gX'] / ep:.1%} g0={base_acc['g0'] / ep:.1%}")
print(f"locked_acc: gX={locked_accu['gX'] / ep:.1%} g0={locked_accu['g0'] / ep:.1%}\n")

print(f'avg_dgX: {avg_dgX / ep:.1%}')
print(f'avg_dg0: {avg_dg0 / ep:.1%}\n')

loss = F.cross_entropy(locked_pred[g0], graph.labels[g0]) - F.cross_entropy(locked_pred[gX], graph.labels[gX])
print(f'loss: {loss}')

In [ ]:
dg0 = (locked_accu["g0"] - base_acc["g0"]) / base_acc["g0"]
dgX = (locked_accu["gX"] - base_acc["gX"]) / base_acc["gX"]

print("==== Accuracies ====")
print(f"         ΔG0\tΔGX")
print(f"task1 | {dg0:.1%}\t{dgX:.1%}")
print()
diff = locked_adj - graph.adj

diffSummary = Metrics.show_metrics(diff, graph.labels, g0, device)
print(diffSummary)

########################
#region Saving perturbations

#from Utils import Export

# if args.save_perturbations != 'N':
#     Export.save_var(f'pertubations-{args.dataset}@{args.ptb_rate}', [best.cpu().numpy().tolist(), g0.cpu().numpy().tolist()], "./perturbations.json")


In [ ]:
"""
args.num_gc_layers = 2
args.mask_act = 'ReLU'
args.mask_bias = False
args.gpu = True #
args.opt = 'adam' # optimizer used
args.lr = args.model_lr # learning rate
args.opt_scheduler = 'step' #

args.opt_decay_step = 0
args.opt_decay_rate = args.reg_epochs # Number of epochs before decay
args.opt_restart = None
args.num_epochs = args.reg_epochs
# Explain Predictions
explainer = Explainer(model=locked_model, adj=locked_adj,
                         feat=graph.features, label=graph.labels,
                         pred=locked_pred, train_idx=graph.idx_train,
                         args=args, graph_mode=False, graph_idx=False)

"""

In [ ]:
"""
idx = Utils.bool_to_idx(g0).detach().cpu().numpy().reshape(1, -1)[0]
print(idx)
explainer.explain_nodes(idx, args)
"""

# Feature Measurements

In [ ]:
## Get indices of (in)correctly predicted nodes ##
base_incorrect = NXG.get_correct_idx(baseline_post_pred, graph.labels, get_correct=False).cpu()
base_correct = NXG.get_correct_idx(baseline_post_pred, graph.labels, get_correct=True).cpu()

locked_incorrect = NXG.get_correct_idx(locked_gcn_cs_post_pred, graph.labels, get_correct=False).cpu()
locked_correct = NXG.get_correct_idx(locked_gcn_cs_post_pred, graph.labels, get_correct=True).cpu()

In [ ]:
print("-- precentage of protected nodes in incorrectly predicted nodes --")
print(f"      Base |  {NXG.protected_node_ratio(base_incorrect, g0):.2%}")
print(f"    Locked |  {NXG.protected_node_ratio(locked_incorrect, g0):.2%}")

In [ ]:
print("-- avg node clustering coefficient --")
print(f"      Base|")
print(f"\t  Correct: {NXG.avg_clustering(graph.nx_g, base_correct):.3f}   |   Label count: {NXG.label_count(base_correct)}")
print(f"\tIncorrect: {NXG.avg_clustering(graph.nx_g, base_incorrect):.3f}   |   Label count: {NXG.label_count(base_incorrect)}\n")

print(f"    Locked|")
print(f"\t  Correct: {NXG.avg_clustering(graph.nx_g, locked_correct):.3f}   |   Label count: {NXG.label_count(locked_correct)}")
print(f"\tIncorrect: {NXG.avg_clustering(graph.nx_g, locked_incorrect):.3f}   |   Label count: {NXG.label_count(locked_incorrect)}\n")

In [ ]:
## 100 epoch avg random-walk
avg_r_walk = NXG.avg_random_walk_protected_ratio(graph.nx_g, g0, walk_len=5, n_walks=100)

print("-- avg random walk protected ratio: (protected nodes visited / total nodes visited) --")
print(f"      Base|")
print(f"\t  Correct: {avg_r_walk[base_correct].sum() / len(base_correct):.5f}")
print(f"\tIncorrect: {avg_r_walk[base_incorrect].sum() / len(base_incorrect):.5f}\n")

print(f".   Locked|")
print(f"\t  Correct: {avg_r_walk[locked_correct].sum() / len(locked_correct):.5f}")
print(f"\tIncorrect: {avg_r_walk[locked_incorrect].sum() / len(locked_incorrect):.5f}\n")

In [ ]:
print("-- avg degree centrality --\n")
print(f"      Base|")
print(f"\t  Correct: {NXG.avg_centrality(graph.nx_g, base_correct):.5f}")
print(f"\tIncorrect: {NXG.avg_centrality(graph.nx_g, base_incorrect):.5f}\n")

print(f"    Locked|")
print(f"\t  Correct: {NXG.avg_centrality(graph.nx_g, locked_correct):.5f}")
print(f"\tIncorrect: {NXG.avg_centrality(graph.nx_g, locked_incorrect):.5f}")

In [ ]:
print("-- avg node degree (post-CS predcitions) --")
print(f"      Base|")
print(f"\t  Correct: {NXG.avg_degree(graph.nx_g, base_correct):.3f}")
print(f"\tIncorrect: {NXG.avg_degree(graph.nx_g, base_incorrect):.3f}\n\n")

print(f"    Locked|")
print(f"\t  Correct: {NXG.avg_degree(graph.nx_g, locked_correct):.3f}")
print(f"\tIncorrect: {NXG.avg_degree(graph.nx_g, locked_incorrect):.3f}\n\n")

In [ ]:
avg_protected_ratio = NXG.protected_neighbor_ratio(graph.nx_g, g0)

print("-- avg ratio degree: (protected neighbors / total neighbors) --")
print(f"      Base|")
print(f"\t  Correct: {avg_protected_ratio[base_correct].sum() / len(base_correct):.5f}")
print(f"\tIncorrect: {avg_protected_ratio[base_incorrect].sum() / len(base_incorrect):.5f}\n")

print(f"    Locked|")
print(f"\t  Correct: {avg_protected_ratio[locked_correct].sum() / len(locked_correct):.5f}")
print(f"\tIncorrect: {avg_protected_ratio[locked_incorrect].sum() / len(locked_incorrect):.5f}\n")

In [ ]:
k_neighborhood_ratio = NXG.k_nearest_neighbor_ratio(graph.nx_g, g0, k=2)

print("-- avg k-neighborhood protected nodes ratio: (protected nodes / total nodes) --")
print(f"  Baseline|")
print(f"\t  Correct: 1-hop: {NXG.avg_k_hop_ratio(base_correct, 1, k_neighborhood_ratio):.5f} \
 |  2-hop: {NXG.avg_k_hop_ratio(base_correct, 2, k_neighborhood_ratio):.5f}")
print(f"\tIncorrect: 1-hop: {NXG.avg_k_hop_ratio(base_incorrect, 1, k_neighborhood_ratio):.5f} \
 |  2-hop: {NXG.avg_k_hop_ratio(base_incorrect, 2, k_neighborhood_ratio):.5f}\n")

print(f"    Locked|")
print(f"\t  Correct: 1-hop: {NXG.avg_k_hop_ratio(locked_correct, 1, k_neighborhood_ratio):.5f} \
 |  2-hop: {NXG.avg_k_hop_ratio(locked_correct, 2, k_neighborhood_ratio):.5f}")
print(f"\tIncorrect: 1-hop: {NXG.avg_k_hop_ratio(locked_incorrect, 1, k_neighborhood_ratio):.5f} \
 |  2-hop: {NXG.avg_k_hop_ratio(locked_correct, 2, k_neighborhood_ratio):.5f}\n")

In [ ]:
stop

# Summarize


In [ ]:
################################################
# Save
################################################
'''
if (args.save == "Y"):
    #import Utils.Export as Export

    def getDiff(location, changeType):
        num = diffSummary[location][changeType]["total"]
        if num == 0: return f"{num:.0f}"
        pct = diffSummary[location][changeType]["same"] / num
        return f"{num:.0f} ({pct:.2%} similar)"

    results = {
        "seed": args.seed,
        "dataset": args.dataset,
        "protect_size": args.protect_size,
        "reg_epochs": args.reg_epochs,
        "ptb_epochs": args.ptb_epochs,
        "ptb_rate": args.ptb_rate,
        "ptb_sample_num": args.num_samples,
        "ptb_sample_size": args.sample_size,
        "ratio_g0": samplingMatrix.g0_ratio.item(),
        "ratio_gX": samplingMatrix.gX_ratio.item(),
        "ratio_g0gX": samplingMatrix.g0gX_ratio.item(),
        "base_g0": baseline_acc["g0"],
        "base_gX": baseline_acc["gX"],
        "d_g0": dg0,
        "d_gX": dgX,
        "edges": diff.abs().sum().item(),
        "add_g0": getDiff("g0", "add"),
        "add_gX": getDiff("gX", "add"),
        "add_g0gX": getDiff("g0gX", "add"),
        "remove_g0": getDiff("g0", "remove"),
        "remove_gX": getDiff("gX", "remove"),
        "remove_g0gX": getDiff("g0gX", "remove"),

    }

    Export.saveData(args.save_location, results)
'''

# Graphs


In [ ]:
#NXG.plot_edge_changes(graph.nx_g, locked_adj, graph.adj)

#NXG.plot_incorrect_subgraph_k_neighborhood(graph.nx_g, postCS_incorrect, k_hops=2, with_labels=True, protected_set=g0)

In [ ]:
#NXG.plot_wrong_pred(nxg, cs_locked_pred, graph.labels)

In [ ]:
#NXG.plot_sets(graph.nx_g, g0)